In [3]:

from Carcassonne_Game.Carcassonne import CarcassonneState
from TicTacToe_Game.TicTacToe import TicTacToeState 
from Function_Optimisation_Game.Function_Optimisation import FunctionOptimisationState
from player.Player import RandomPlayer
from player.MCTS_Player import MCTSPlayer
from player.MCTS_RAVE_Player import MCTS_RAVEPlayer
from player.MCTS_ES_BACK_Player import MCTS_ES_BACK_Player
from player.MCTS_ES_BACK_SEM_Player import MCTS_ES_BACK_SEM_Player

import Experimental_Setup as exps
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#from plotly.tools import make_subplots

import os
import pandas as pd
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import numpy as np
import ast
import random
import math
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import bernoulli
from itertools import repeat


In [ ]:
### Generate and save analysis plots for all functions with one dimension
join = "/"
logs_path = "logs/FO"
for fn in range(5):
   print(fn)
   for n_splits in [2,3]:
      print(n_splits)
      max_depth = 4
      if n_splits == 3:
         max_depth -= 1
      if fn in [3,4]:
         max_depth += 2
      dummy_state = FunctionOptimisationState(players=[None], function=fn, ranges=[[0,1]], splits=n_splits)
      plot = exps.fo_function_analysis(dummy_state, max_depth=max_depth)
      #plot.write_image(os.path.join(logs_path, "f" + str(fn) + "s" + str(n_splits) + 'analysis.png'), width=800, height=800)
      plot.show()

In [4]:
#Functoin analysis

max_depths = [4,5,5,6,6]
splits = 2
for f_index in range(1,2):
   max_depth = max_depths[f_index]
   max_depth = 0
   random_player = RandomPlayer()
   dummy_state = FunctionOptimisationState(players=[random_player], function=f_index, ranges=[[0,1],[0,1]], minimum_step=0.00001, splits=splits)
   title_plot = "Function " + str(f_index) + " expected rewards"
   fig = exps.fo_function_analysis(dummy_state, title_plot, max_depth=max_depth, max_val=dummy_state.max_x[f_index])
   fig.show()
   #fig.write_image(os.path.join('logs', "Old", "FO", "f" + str(f_index) + "s" + str(splits) + "d" + str(max_depth) + '.png'))

In [ ]:
def fo_function_analysis2(fo_state, max_depth=3, print_logs=False):
   """Returns a figure with a 2d histogram plotting the function landscape as MCTS will se it. Manual (Fast)"""

   #Find evaluation points
   stop = {}
   start = {}
   max_depth_step = {}
   x = {}
   division_size = {}
   dimensions = len(fo_state.ranges)

   #splits by dimension
   for d in range(dimensions):
      
      stop[d] = fo_state.ranges[d][1]
      start[d] = fo_state.ranges[d][0]
      max_depth_step[d] = (stop[d]-start[d])/(fo_state.splits**max_depth)
      division_size[d] = stop[d] - start[d]
      while division_size[d] > fo_state.minimum_step:
         division_size[d] = division_size[d]/fo_state.splits   
      #if print_logs: print("dimension", d, " division_size", division_size[d], "start", start[d], "stop", stop[d], "max_detph_step", max_depth_step[d])

      #get central points
      x[d] = []   
      next_start = start[d]
      center_distance = division_size[d]/2
      while next_start+center_distance < stop[d]:
         next_stop = next_start + division_size[d]
         x[d].append(next_start+center_distance) #gets the value in the middle
         next_start = next_stop
      #if print_logs: print("x", x[d])
   #if print_logs: print("max_depth_step", max_depth_step)


   fig_x = {}
   fig_y = {}
   fig_z = {}
   for current_depth in reversed([md+1 for md in range(max_depth)]):
      depth_step = {}
      for d in range(dimensions):
         depth_step[d] = (stop[d]-start[d])/(fo_state.splits**current_depth)
      #if print_logs: print("current_depth", current_depth, "depth_step", depth_step)

      if max_depth == current_depth:
         granular_x = x[0]
         granular_y = x[1]
      else:
         granular_x = fig_x[current_depth+1]
         granular_y = fig_y[current_depth+1]
         granular_z = {}
         for i in range(len(granular_x)):
            granular_z[(granular_x[i], granular_y[i])] = fig_z[current_depth+1][i]

      all_depth_steps = {}
      for d in range(dimensions):
         all_depth_steps[d] = [[i*depth_step[d], (i+0.5)*depth_step[d], (i+1)*depth_step[d]] for i in range(fo_state.splits**current_depth)]
         all_depth_steps[d][0][0] = start[d]
         all_depth_steps[d][-1][2] = stop[d]
      #if print_logs: print("all_depth_steps shape:", str([str(k)+":"+str(len(v)) for k,v in all_depth_steps.items()]))
      #if print_logs: print("all_depth_steps max:", str(max([max(v) for k,v in all_depth_steps.items()])), str(min([min(v) for k,v in all_depth_steps.items()])))

      avg_by_y={}
      for j in granular_y:
         avg_by_y[j] = {}
         steps = 0
         count = 0
         accum = 0
         for i in granular_x:
            #if print_logs: print("i", i, "steps", steps)
            if i > all_depth_steps[0][steps][2]:
               avg_by_y[j][all_depth_steps[0][steps][1]] = accum/count
               count = 0
               accum = 0
               steps += 1
            if max_depth == current_depth:
               accum = accum + fo_state.function([i,j])
            else:
               accum = accum + granular_z[(i,j)]
            count += 1
         #if print_logs: print("i", i, "steps", steps)
         avg_by_y[j][all_depth_steps[0][steps][1]] = accum/count
         all_x_keys = avg_by_y[j].keys()
      #if print_logs: print("avg_by_y", avg_by_y)
      #if print_logs: print("all_x_keys", all_x_keys)

      fig_x[current_depth]=[]
      fig_y[current_depth]=[]
      fig_z[current_depth]=[]
      for i in all_x_keys:
         steps = 0
         count = 0
         accum = 0
         for j in avg_by_y.keys():
            if j > all_depth_steps[1][steps][2]:
               fig_x[current_depth].append(i)
               fig_y[current_depth].append(all_depth_steps[1][steps][1])
               fig_z[current_depth].append(accum/count)
               count = 0
               accum = 0
               steps += 1
            accum = accum + avg_by_y[j][i]
            count += 1
         fig_x[current_depth].append(i)
         fig_y[current_depth].append(all_depth_steps[1][steps][1])
         fig_z[current_depth].append(accum/count)


   #create subplots
   plot_pixels = 150
   n_plots = max_depth
   """
   if n_plots%2==0:
      row_heights = [1/(n_plots/2) for _ in range(int(n_plots/2))]
      column_widths = [0.5, 0.5]
   else:
   """
   row_heights = [1/n_plots for _ in range(n_plots)]
   column_widths = [1]
   #if print_logs: 
   #   print(row_heights, column_widths)
   #   print("rows",len(row_heights),"cols",len(column_widths))
   fig = make_subplots(
      rows=len(row_heights)
      ,cols=len(column_widths)
      ,shared_xaxes=True
      ,vertical_spacing=0.03
      ,row_heights = row_heights
      ,column_widths = column_widths
      #,specs=[[{"secondary_y": True}] for _ in range(len(column_widths))]
      )
   
   #add function plot
   #x = np.linspace(0.001,1,5000)
   #y = [fo_state.function([i]) for i in x]
   #fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=1,col=1)
   

   #add analysis plots
   n_ticks_colorbar = [0,3,0,0,0,0,0,0,0,0,0,0]
   for d in range(1,max_depth+1):
      if d==1: show_legend = True
      else: show_legend = False
      #print("n_bins", str(fo_state.splits**d))
      #if print_logs:
         #print("fig_x[d]", fig_x[d])
         #print("fig_y[d]", fig_y[d])
         #print("fig_z[d]", fig_z[d])
         #print("nbinsx",str(fo_state.splits**d))
      fig.add_trace(
         go.Histogram2d(x=fig_x[d], y=fig_y[d], z=fig_z[d],histfunc ="avg"
            #,autobinx =False
            #,nbinsx=5
            ,xbins = {"size":(stop[0]-start[0])/fo_state.splits**d}
            ,ybins = {"size":(stop[1]-start[1])/fo_state.splits**d}
            #,nbinsy=fo_state.splits**d
            #,color_continuous_scale="gray"
            ,colorscale = [[0, 'rgb(235,235,235)'], [1, 'rgb(0,0,0)']]
            ,showlegend = False
            ,colorbar = {"nticks":n_ticks_colorbar[d]}
            #,texttemplate= "%{z}"
            )
         ,row=d,col=1)
      #fig.add_trace(go.Scatter(x=[start,stop], y=[max(y),max(y)], line=dict(color='royalblue', width=2, dash='dash'),showlegend=False,marker={"color":"blue"}),row=d+1,col=1)

   #update fig layout
   #fig.update_layout(barmode='stack')
   fig.update_layout(margin=dict(l=10, r=10, t=30, b=20)
      ,width=plot_pixels+100
      ,height=plot_pixels*n_plots
      ,autosize=False
      ,plot_bgcolor='rgba(0,0,0,0)',title={"text":"2D Function analysis"}
                #,legend=dict(
                    #title = "Formula",
                    #orientation="h",
                    #yanchor="top",
                    #y=-0.65,
                    #xanchor="center",
                    #x=0.5,  
                    #font = dict(family = "Arial", size = 14, color = "black"),
                    #bordercolor="LightSteelBlue",
                    #borderwidth=2,
                    #itemsizing='trace',
                    #itemwidth = 30
                    #)  
                    )
   #fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black')
   #fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   #fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   # fig.update_xaxes(range=[start,stop])
   fig.update_xaxes(range=[start[0],stop[0]])
   fig.update_yaxes(range=[start[1],stop[1]])
   

   return fig


def hist2d(fo_state, max_depth=3, print_logs=False):
   """Returns a figure with a 2d histogram plotting the function landscape as MCTS will se it. Using the library (slow)"""
   stop = {}
   start={}
   x = {}
   division_size = {}
   dimensions = len(fo_state.ranges)

   #splits by dimension
   for d in range(dimensions):
      if print_logs: print("dimension", d)
      stop[d] = fo_state.ranges[d][1]
      start[d] = fo_state.ranges[d][0]
      division_size[d] = stop[d] - start[d]
      while division_size[d] > fo_state.minimum_step:
         division_size[d] = division_size[d]/fo_state.splits
      if print_logs: print("division_size", division_size[d])

      #get central points
      x[d] = []   
      next_start = start[d]
      center_distance = division_size[d]/2
      while next_start < stop[d]:
         next_stop = next_start + division_size[d]
         x[d].append(next_start+center_distance) #gets the value in the middle
         next_start = next_stop
      #if print_logs: print("x", x[d])

   #arrange data
   fig_x = []
   fig_y=[]
   fig_z=[]
   for i in x[0]:
      for j in x[1]:
         fig_x.append(i)
         fig_y.append(j)
         fig_z.append(fo_state.function([i,j]))

   if print_logs:
      print("len(fig_x)", len(fig_x), max(fig_x))
      print("len(fig_y)", len(fig_y), max(fig_y))
      print("len(fig_z)", len(fig_z), max(fig_z))

   fig = go.Figure(go.Histogram2d(x=fig_x, y=fig_y, z=fig_z,histfunc ="avg"
      ,nbinsx=fo_state.splits**max_depth
      ,nbinsy=fo_state.splits**max_depth
      #color_continuous_scale="Viridis",
      ,texttemplate= "%{z}"))

   return fig
   
max_depths = [4,5,5,6,6]
splits = 2
for f_index in range(5):
   max_depth = max_depths[f_index]
   random_player = RandomPlayer()
   dummy_state = FunctionOptimisationState(players=[random_player], function=f_index, ranges=[[0,1],[0,1]], minimum_step=0.001, splits=splits)
   title_plot = "Function " + str(f_index) + " analysis"
   fig = exps.fo_function_analysis(dummy_state, title_plot, max_depth=max_depth, max_val=dummy_state.max_x[f_index])
   fig.show()
   #fig.write_image(os.path.join('logs', "Old", "FO", "f" + str(f_index) + "s" + str(splits) + "d" + str(max_depth) + '.png'))

In [ ]:
#Collect logs
join = "/"
logs_path = "logs/FO"
output_name = "collective_logs.csv"
exp_names = [ item for item in os.listdir(logs_path) if os.path.isdir(os.path.join(logs_path, item)) ]
final_df = pd.DataFrame()
for i,exp_name in enumerate(exp_names):
   data = pd.read_csv(logs_path + join + exp_name + join + "Final_Player_logs.csv")
   pars = pd.read_csv(logs_path + join + exp_name + join + "Parameter_logs.csv")
   fi_list = [pars["func_index"][0] for _ in range(len(data))]
   c_list = [pars["c_param"][0] for _ in range(len(data))]
   expname_list = [exp_name for _ in range(len(data))]
   data["function"] = fi_list
   data["c"] = c_list
   data["expname"] = expname_list
   if i==0:
      final_df = pd.DataFrame(data)
   else:
      final_df = pd.concat([final_df,data]) 
   
final_df.to_csv(logs_path + join + output_name, index=False)

In [ ]:
"""
Collects data in "collyective_tree_logs.csv".
Logs names should be saved as "Results_f0_c0.5" where f is the function, c is the parameter
"""

join = "/"
logs_path = "logs/FO"
output_name = "collective_tree_logs.csv"
final_df = pd.DataFrame()
all_data = []
all_f = []
all_c = []
exp_names = [item for item in os.listdir(logs_path) if os.path.isdir(os.path.join(logs_path, item))]
for i,exp_name in enumerate(exp_names):
   f_index = int(exp_name.split("_f")[-1][0])
   all_f.append(f_index)
   c = float(exp_name.split("_c")[-1])
   all_c.append(c)
   data = pd.read_csv(logs_path + join + exp_name + join + "Tree_data.csv")
   data["c_param"] = [c for _ in range(len(data))]
   data["f_index"] = [f_index for _ in range(len(data))]
   all_data.append(data)
all_f = set(all_f)
all_c = set(all_c)
final_df = pd.concat(all_data)
final_df.to_csv(logs_path + join + output_name, index=False)

In [ ]:
###  HISTOGRAM PLOT GENERATION

#https://stackoverflow.com/questions/39499173/plotly-changing-the-line-styles
def show_search(data_list, function, title, divisions, n_buckets = 100, type="divisions", subplot_titles=None, max_x_location = None, y_ref_value = None):
   #print("received subplot_titles", subplot_titles)
   
   #print("edited subplot_titles", subplot_titles)
   if divisions in [2,3]: colors = ["#696969","#4f4f4f","#2f2f2f"]
   if divisions==4: colors = ["#cf0000","#a2000d","#740017","#53001b"]
   n_plots = len(data_list)
   even_spaces = 1/(n_plots+1)
   row_heights = [even_spaces for _ in range(n_plots)] + [even_spaces]
   fig = make_subplots(rows=n_plots+1, cols=1,shared_xaxes=True,vertical_spacing=0.03,row_heights=row_heights
                        ,subplot_titles = subplot_titles, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)])
   if function is not None:
      x = np.linspace(0.001,1,5000)
      y = [function([i]) for i in x]
      fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=n_plots+1,col=1)

   show_legend = [False] + [False for _ in range(n_plots)]
   for i,data in enumerate(data_list):
      for div in range(divisions):
         if div%divisions == 0: s1 = "{:2.0f}".format(100*(div/divisions))
         else: s1 = "{:2.1f}".format(100*(div/divisions))
         if (div+1)%divisions == 0: s2 = "{:2.0f}".format(100*((div+1)/divisions))
         else: s2 = "{:2.1f}".format(100*((div+1)/divisions))

         div_name = s1 + "% to " + s2 + "%"
         temp_data = data.loc[data["id_block"]==div]
         fig.add_trace(go.Histogram(x=temp_data.x, nbinsx=n_buckets, xbins={"start":0,"end":1,"size":1/n_buckets}
                  #, legendgroup=div, name=div_name
                  , showlegend=show_legend[i]
                  , marker={"color":colors[div]}),row=i+1,col=1)
         #fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=i+1,col=1,secondary_y=True)

   #fig.update_layout(barmode='stack')

   fig.update_layout(margin=dict(l=10, r=10, t=30, b=20),width=800,height=800,plot_bgcolor='rgba(0,0,0,0)',title={"text":title}
               ,barmode='stack'
               ,legend=dict(
                    #title = "Formula",
                    #orientation="h",
                    #yanchor="top",
                    y=-0.65,
                    xanchor="center",
                    x=0.5,  
                    font = dict(family = "Arial", size = 14, color = "black"),
                    #bordercolor="LightSteelBlue",
                    borderwidth=2,
                    itemsizing='trace',
                    itemwidth = 30
                    )  )
   #fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black')
   fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, showgrid=True, nticks=5, gridcolor="red", gridwidth=0.1)#, dtick=1000, showgrid=True, gridcolor="red", gridwidth=0.1)#, nticks=10, showgrid=True, griddash="dash", gridwidth=0.5, gridcolor="LightPink")
   #fig.update_layout(yaxis4=dict(dtick=500, showgrid=True, gridcolor="red", gridwidth=0.1))

   list_of_shapes = []
   if max_x_location is not None:
      for subplot_n in range(1,n_plots+2):
         yref = "y" + str(subplot_n*2)
         list_of_shapes.append({'type': 'line','y0':0,'y1': 1,'x0':max_x_location, 
                                    'x1':max_x_location,'xref':"x",'yref':yref,
                                    'line': {'color': 'red','width': 1}})
   if y_ref_value is not None:
         for subplot_n in range(1,n_plots+1):
            yref = "y" + str(subplot_n*2-1)
            list_of_shapes.append({'type': 'line','y0':y_ref_value,'y1': y_ref_value,'x0':0, 
                                       'x1':1,'xref':"x",'yref':yref,
                                       'line': {'color': 'red','width': 1, "dash":"dash"}})
   if list_of_shapes != []:
      fig['layout'].update(shapes=list_of_shapes)
      """
      
      for subplot_n in range(1, n_plots+2):
         yref = "y" + str(subplot_n)
         fig.add_shape(go.layout.Shape(type="line", yref=yref, xref="x", x0=max_x_location, x1 = max_x_location, y0=0, y1=1, line=dict(color="red", width=1),),row=subplot_n, col=1)
      """
   #fig.show()
   return fig

def show_search2(data_list, function, title, divisions, n_buckets = 100, type="divisions", subplot_titles=None, domain=[0,1]):

   #calc variables
   if divisions in [2,3]: colors = ["#696969","#4f4f4f","#2f2f2f"]
   n_plots = len(data_list)
   bucket_size = (domain[1] - domain[0])/n_buckets
   even_spaces = 1/(n_plots+1)
   row_heights = [even_spaces for _ in range(n_plots)] + [even_spaces]
   show_legend = [False] + [False for _ in range(n_plots)]

   #create fig
   fig = make_subplots(rows=n_plots+1, cols=1,shared_xaxes=True,vertical_spacing=0.03,row_heights=row_heights
                        ,subplot_titles = subplot_titles, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)])
   
   #add function plot
   if function is not None:
      x = np.linspace(0.001,1,5000)
      y = [function([i]) for i in x]
      fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=n_plots+1,col=1)

   for i,data in enumerate(data_list):
      #print("data index ", str(i))
      #calc data for histogram
      new_df = pd.DataFrame({"x":[],"id_block":[]})

      previous_max_depth = [0 for _ in range(n_buckets)]
      previous_mean_depth = [0 for _ in range(n_buckets)]
      for div in range(divisions):

         #division names
         if div%divisions == 0: s1 = "{:2.0f}".format(100*(div/divisions))
         else: s1 = "{:2.1f}".format(100*(div/divisions))
         if (div+1)%divisions == 0: s2 = "{:2.0f}".format(100*((div+1)/divisions))
         else: s2 = "{:2.1f}".format(100*((div+1)/divisions))
         div_name = s1 + "% to " + s2 + "%"

         #calc max depth
         temp_data = data.loc[data["id_block"]==div]
         #print("data in division ", str(div)," : ", str(len(temp_data)))
         for b in range(n_buckets):
            bucket_center = b*bucket_size + bucket_size/2
            #subset = temp_data.loc[((temp_data["x"] < bucket_size*(b+1)) & (temp_data["x"]) >= bucket_size*b)]
            tset = temp_data.loc[(temp_data["x"] >= bucket_size*(b))]
            subset = tset.loc[(tset["x"] < bucket_size*(b+1))]
            #print("data in subset ", str(b)," : ", str(len(subset)))
            if len(subset) > 0:
               max_depth = max(subset["feature_0"])
               mean_depth = int(math.floor(stats.mean(subset["feature_0"])))
               #print("max_depth: ", str(max_depth))
               copies = max_depth - previous_max_depth[b]
               #copies = mean_depth - previous_max_depth[b]
               #print("copies: ", str(copies))
               if copies > 0:
                  previous_max_depth[b] = max_depth
                  previous_mean_depth[b] = mean_depth
                  subset_dict = {"x":[bucket_center for _ in range(copies)], "id_block":[div for _ in range(copies)]}
                  subset_df = pd.DataFrame(subset_dict)
                  new_df = pd.concat([new_df, subset_df])
                  #print("df len after added: ", str(len(new_df)))

      #new_df.to_csv(os.path.join("logs","testdf.csv"), index=False)
      #add histogram plot
      #print(divisions, "divisions")
      for div in range(divisions):
         final_data = new_df.loc[new_df["id_block"]==div]
         #print("r",str(i+1))
         fig.append_trace(go.Histogram(x=final_data.x, nbinsx=n_buckets, xbins={"start":0,"end":1,"size":1/n_buckets}, legendgroup=div, name=div_name, showlegend=False, marker={"color":colors[div]}),row=i+1,col=1)
         #fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=i+1,col=1,secondary_y=True)

   #update fig layout
   fig.update_layout(barmode='stack')
   fig.update_layout(margin=dict(l=10, r=10, t=30, b=20),width=800,height=800,plot_bgcolor='rgba(0,0,0,0)',title={"text":title}
                ,legend=dict(
                    #title = "Formula",
                    #orientation="h",
                    #yanchor="top",
                    y=-0.65,
                    xanchor="center",
                    x=0.5,  
                    font = dict(family = "Arial", size = 14, color = "black"),
                    #bordercolor="LightSteelBlue",
                    borderwidth=2,
                    itemsizing='trace',
                    itemwidth = 30
                    )  )
   #fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black')
   fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   #fig.show()
   return fig

def show_search_depth(data_list):
   #agents need to be re-ordered (no hardcode order)
   colors = ["#696969","#4f4f4f","#2f2f2f","#000000","#0f0f0f","#666666","#234654"]
   markers = ["diamond","cross","x","triangle-up","triangle-down","star","star-square","corcle-cross","square", "pentagon"]
   fig = go.Figure()
   for data_idx, data in enumerate(data_list):#list(reversed(data_list)):
      name = data["player"].unique()[0]
      if len(name) > 12: name = name[:12]
      print(len(data))
      #temp_data = data.groupby(by="id").mean()
      #print(len(temp_data))
      x = data["id"].unique()
      x = [k for k in x if k%100==0]
      #y = [stats.mean(data.loc[data["id"]==id,["feature_0"]]["feature_0"]) for id in x]
      y=[]
      for id in x:#list(reversed(x)):
         temp = data.loc[data["id"]==id,["feature_0"]]["feature_0"]
         #print("filtered_len",len(temp))
         y.append(stats.mean(temp))
      fig.add_trace(go.Scatter(x=x, y=y
            ,showlegend=True
            ,name=name
            ,mode='lines+markers'#"markers"
            ,marker_symbol = markers[data_idx]# + "-open-dot"
                        #,marker_color = "black"
                        #,marker_size=9
                        ,marker=dict(
                            color=colors[data_idx]#"black",#"red",#'rgba(135, 206, 250, 0.5)',
                            ,size=9
                            ,line=dict(
                                #color='MediumPurple',
                                width=0.5)
                            )))#,marker_color = "black"))
   fig.update_layout(
                #title_text=title
                #,title_x=0.5
                #,title_y=1
                #xaxis_title="Turn"
                #,yaxis_title="Nodes"
                autosize=False
                ,width=700
                ,height=350
                ,plot_bgcolor='rgba(0,0,0,0)'
                ,legend=dict(
                    #title = "Formula",
                    #orientation="h",
                    #yanchor="top",
                    y=0,
                    #xanchor="center",
                    x=0.7,  
                    font = dict(family = "Arial", size = 14, color = "black"),
                    #bordercolor="LightSteelBlue",
                    borderwidth=2,
                    itemsizing='trace',
                    itemwidth = 30
                    )  
                )
   fig["layout"]["yaxis"]["tickmode"] = "linear"
   fig["layout"]["yaxis"]["tick0"] = 4
   fig["layout"]["yaxis"]["dtick"] = 2
   fig["layout"]["yaxis"]["showgrid"] = True
   fig["layout"]["yaxis"]["gridcolor"] = "black"
   fig["layout"]["yaxis"]["gridwidth"] = 0.6
   fig["layout"]["xaxis"]["tickmode"] = "linear"
   fig["layout"]["xaxis"]["tick0"] = 0
   fig["layout"]["xaxis"]["dtick"] = 500
   fig["layout"]["xaxis"]["showgrid"] = True
   fig["layout"]["xaxis"]["gridcolor"] = "black"
   fig["layout"]["xaxis"]["gridwidth"] = 0.6
   fig['layout'].update(margin=dict(
                                       l=20,
                                       r=10,
                                       b=0,
                                       t=10))    
   fig.update_xaxes(range=[0,5000])
   fig.update_yaxes(range=[4,18])
   return fig

def get_subset(data, agent_name, f_index, c_param=None):
   temp_data = data.loc[data["f_index"]==f_index]
   if c_param is not None:
      temp_data = temp_data.loc[temp_data["c_param"]==c_param]
   #print(temp_data["player"].unique())
   temp_data = temp_data.loc[temp_data["player"] == agent_name]
   return temp_data

dummy_state = FunctionOptimisationState(players=[None], function=0, ranges=[[0,1]], splits=2)
functions = dummy_state.function_list
join = "/"
logs_path = "logs/FO"
output_name = "collective_tree_logs.csv"
data = pd.read_csv(logs_path + join + "collective_tree_logs.csv")
n_buckets = 500
exp = 1
f_max_locations = [0.5,0.867,None,0.1,0.1]
#Print parameters
#1.4142135623730951
if exp == 0:
   for f_index in [0,1,2,3,4]:
      agents_names = data["player"].unique()
      c_params = data["c_param"].unique()
      generic_name = "MCTS_c"
      agents_names = [x for x in agents_names if generic_name in x]
      names = []
      data_list = []
      for c in c_params:
         string_c = str(c)
         if string_c.split(".")[-1] == "0":
            string_c = string_c[:-2]
         names.append(generic_name + string_c) 
      for name in names:
         temp_data = get_subset(data, name, f_index)
         data_list.append(temp_data)
         #print(name)
         #print(len(temp_data))
      #print(len(data_list))
      #for d in data_list: print(len(d))
      #print(names)
      plot = show_search(data_list, functions[f_index], "", 3, n_buckets = n_buckets, type="divisions", subplot_titles = agents_names+["Function "+str(f_index)])
      #plot.write_image(os.path.join(logs_path, "MCTS_f" + str(f_index) + '.png'), width=800, height=800)
      plot.show()
#
if exp == 1:
   for f_index in [0,1,2,3,4]:
      agents_names = data["player"].unique()
      print(agents_names)
      c_params = data["c_param"].unique()
      c_params.sort()
      generic_name = "MCTS_c"
      agents_names = [x for x in agents_names]
      names = []
      data_list = []

      #order names
      for c in c_params:
         string_c = str(c)
         if string_c.split(".")[-1] == "0":
            string_c = string_c[:-2]
         names.append(generic_name + string_c)
      for name in agents_names:
         if "SE_MCTS" in name:
            names.append(name)
      for name in names:
         temp_data = get_subset(data, name, f_index)
         data_list.append(temp_data)
      
      treated_names = []
      for it,st in enumerate(names):
         new_string = st.replace("_c"," C = ")
         new_string = new_string.replace("1.414","sqrt(2)fred")
         new_string = new_string.split("fred")[0]
         if "SE_MCTS" in new_string:
            if "2600" in new_string:
               new_string = "SE_MCTS partial simulations"
            else:
               new_string = "SE_MCTS complete simulations"
         treated_names.append(new_string)
      
      
      #print(len(data_list))
      #for d in data_list: print(len(d))
      #print(names)
      plot = show_search(data_list, functions[f_index], "", 3, n_buckets = n_buckets, type="divisions", subplot_titles = treated_names+["Function "+str(f_index+1)], max_x_location=f_max_locations[f_index], y_ref_value=None)
      #plot.show()
      plot.write_image(os.path.join(logs_path, "f" + str(f_index) + '_v2.png'), width=800, height=800)
      #depth_plot = show_search_depth(data_list)
      #depth_plot.show()
      #depth_plot.write_image(os.path.join(logs_path, "depth" + str(f_index) + '.png'), width=600, height=350)
      #plot = show_search2(data_list, functions[f_index], "", 3, n_buckets = 200, type="divisions", subplot_titles = agents_names+["Function "+str(f_index)])
      plot.show()
      #depth_plot.write_image(os.path.join(logs_path, "depth_f" + str(f_index) + '.png'), width=800, height=400)
      #plot.write_image(os.path.join(logs_path, "depth_histo_max_2k_f" + str(f_index) + '.png'), width=800, height=800)

In [ ]:
i=0
divisions=3
n_buckets = 100
domain=[0,1]
bucket_size = (domain[1] - domain[0])/n_buckets

data = data_list[i]
div = 0
temp_data = data.loc[data["id_block"]==div]
b=70
bucket_center = b*bucket_size + bucket_size/2
testsubset = temp_data.loc[((temp_data["x"] < bucket_size*(b+1)) & (temp_data["x"]) >= bucket_size*b)]
print(bucket_size*(b+1))
print(bucket_size*(b))
#print(min(temp_data["x"]))
subset = temp_data.loc[(temp_data["x"] >= bucket_size*(b))]
subsubset = subset.loc[(subset["x"] < bucket_size*(b+1))]
print("data in subset ", str(b)," : ", str(len(subset)))
print("data in subsubset ", str(b)," : ", str(len(subsubset)))
print("data in testsubset ", str(b)," : ", str(len(subset)))
print(min(subsubset["x"]))
print(max(subsubset["x"]))
print(min(testsubset["x"]))
print(max(testsubset["x"]))


0.71
0.7000000000000001
data in subset  70  :  14510
data in subsubset  70  :  1715
data in testsubset  70  :  14510
0.7000045776367188
0.7099609375
0.03125
0.7099609375


In [ ]:
#https://plotly.com/python/mapbox-density-heatmaps/
#https://plotly.com/python/v3/density-plots/
#https://plotly.com/python/imshow/
#https://plotly.com/python/mapbox-density-hea
#https://plotly.com/python/histograms/
#https://plotly.com/python/subplots/

#function options
def func(x): 
   return 0.5 + 0.5*math.sin(20*x[0])
def func2(x): 
   return (0.5+0.5*math.sin(20*x[0]))*(0.5+0.5*math.sin(20*x[1]))
def func3(x): return x*x
def func4(x): return abs(math.sin(1/math.pow(x[0]-0.5,1)))
def func5(x): #from the paper
   if x[0] < 0.5:
      return 0.5+0.5*abs(math.sin(1/pow(x[0],5)))
   else:
      return 7/20+0.5*abs(math.sin(1/pow(x[0],5)))
def func6(x): return 0.37*pow(math.sin(12*x[0]),20)*1/(0.01+math.exp(x[0]-1))+x[0]/2
def func7(x): return 0.2*pow(math.sin(30*x[0]),70)*1/(0.06+math.exp(x[0]-2))+pow(x[0],2)/2
def func8(x): return pow(math.sin(31*x[0]),10)*0.18/(0.04+math.exp(x[0]-2))+pow(x[0],2)/2
def func9(x): return (-0.7*x[0]+1)*pow(math.sin(25*x[0]),60)+pow(x[0],2)/1.5
def func10(x): return (-0.6*x[0]+1)*pow(math.sin(25*x[0]),60)+pow(x[0],2)/2
def func11(x): return (-0.6*x[0]+1)*pow(math.sin(31*x[0]),10)+pow(x[0],2)/2 #10 picos
def func12(x): 
   """10 peaks deceptive"""
   return (0.4*x[0])+(-0.6*x[0]+1)*pow(math.sin(10*math.pi*x[0]),40)
def func_p1(x): 
   """Unimodal, centered"""
   return math.sin(math.pi*x[0])
def func_p1(x): 
   """Multimodal, centered"""
   return math.sin(math.pi*x[0])
def f0(x):
   """Unimodal, centered"""
   return math.sin(math.pi*x[0])
def f1(x):
   """Multimodal, paper bubeck"""
   return 0.5*(math.sin(13*x[0])*math.sin(27*x[0])+1)
def f2(x):
   """Smoothness with levels, paper finnsson"""
   if x[0] < 0.5:
      return 0.5+0.5*abs(math.sin(1/pow(x[0],5)))
   else:
      return 7/20+0.5*abs(math.sin(1/pow(x[0],5)))
def f3(x):
   """Deceptive"""
   return (0.5*x[0])+(-0.7*x[0]+1)*pow(math.sin(5*math.pi*x[0]),4)
def f4(x):
   """Deceptive, search traps"""
   return (0.5*x[0])+(-0.7*x[0]+1)*pow(math.sin(5*math.pi*x[0]),80)
function_list=[f0,f1,f2,f3,f4]

paper_funcs = [func_p1,func12, func4, func5]

def tree_data(player, divisions=3, dimension=0, early_cut=False):
   """Division method Options: percentage, best_changed
      early_cut: only nodes that were added before a terminal state was reached by an expanded node"""
   
   if early_cut:
      nodes = []
      for key,node in player.nodes_dict.items():
         if node.state.isGameOver:
            break
         nodes.append(node)
   else:
      nodes = [node for key,node in player.nodes_dict.items()]
   n_nodes = len(player.nodes_dict)

   id_list = []
   x_list = []
   id_block_list = []
   features_list = []

   for node in nodes:
      id_list.append(node.id)
      x_list.append(node.state.eval_point()[dimension])
      id_block_list.append(int((node.id/(n_nodes+1))/(1/divisions)))
      features_list.append(node.state.featureVector())
   data_dict = {"player":player.name,"id":id_list, "x":x_list, "id_block":id_block_list}

   for feature_index in range(len(features_list[0])): #Assumes the feature vector size is constant
      data_dict["feature_"+str(feature_index)] = [fv[feature_index] for fv in features_list]

   data = pd.DataFrame(data_dict)
   return data

def best_tree_path(node, recommendation_policy="reward"):
   if recommendation_policy == "reward":
      while node.child != []:
         if node.playerSymbol == 1:
            node = sorted(node.child, key = lambda c: c.Q)[-1]
         else:
            node = sorted(node.child, key = lambda c: c.Q)[0]
   elif recommendation_policy == "visits":
      while node.child != []:
         if node.playerSymbol == 1:
            node = sorted(node.child, key = lambda c: c.visits)[-1]
         else:
            node = sorted(node.child, key = lambda c: c.visits)[0]
   return node

def show_search(data_list, function, title, divisions, n_buckets = 100, type="divisions"):
   if divisions in [2,3]: colors = ["#5e4e9c","#4169b0","#009ee3"]
   if divisions==4: colors = ["#cf0000","#a2000d","#740017","#53001b"]
   n_plots = len(data_list)
   even_spaces = 1/(n_plots+1)
   row_heights = [even_spaces for _ in range(n_plots)] + [even_spaces]
   fig = make_subplots(rows=n_plots+1, cols=1,shared_xaxes=True,vertical_spacing=0.05,row_heights=row_heights)

   show_legend = [False] + [False for _ in range(n_plots)]
   for i,data in enumerate(data_list):
      for div in range(divisions):
         if div%divisions == 0: s1 = "{:2.0f}".format(100*(div/divisions))
         else: s1 = "{:2.1f}".format(100*(div/divisions))
         if (div+1)%divisions == 0: s2 = "{:2.0f}".format(100*((div+1)/divisions))
         else: s2 = "{:2.1f}".format(100*((div+1)/divisions))

         div_name = s1 + "% to " + s2 + "%"
         temp_data = data.loc[data["id_block"]==div]
         fig.append_trace(go.Histogram(x=temp_data.x, nbinsx=n_buckets, xbins={"start":0,"end":1,"size":1/n_buckets}, legendgroup=div, name=div_name, showlegend=show_legend[i], marker={"color":colors[div]}),row=i+1,col=1)
         

   fig.update_layout(barmode='stack')

   if function is not None:
      x = np.linspace(0.001,1,5000)
      y = [function([i]) for i in x]
      fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=n_plots+1,col=1)


   fig.update_layout(margin=dict(l=10, r=10, t=30, b=20),width=1000,height=800,plot_bgcolor='rgba(0,0,0,0)',title={"text":title}
                ,legend=dict(
                    #title = "Formula",
                    #orientation="h",
                    #yanchor="top",
                    y=-0.65,
                    xanchor="center",
                    x=0.5,  
                    font = dict(family = "Arial", size = 14, color = "black"),
                    #bordercolor="LightSteelBlue",
                    borderwidth=2,
                    itemsizing='trace',
                    itemwidth = 30
                    )  )
   fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black')
   fig.show()
   return fig

def show_2d_search(data_list, function, title, divisions, n_buckets = 100):
   colors = ["#5e4e9c","#4169b0","#009ee3"]
   n_plots = len(data_list)
   even_spaces = 1/(n_plots+1)
   row_heights = [even_spaces for _ in range(n_plots)] + [even_spaces]
   fig = make_subplots(rows=n_plots+1, cols=1,shared_xaxes=True,vertical_spacing=0.05,row_heights=row_heights)

   show_legend = [False] + [False for _ in range(n_plots)]
   for i,data in enumerate(data_list):
      #temp_data = data.assign(div = lambda x: int((x.id/(len(data)+1))/(1/divisions)))
      div_list = [int((x["id"]/(max(data["id"])+1))/(1/divisions)) for i,x in data.iterrows()]
      #print("max div",max(div_list))
      temp_data = data.assign(div = div_list)
      fig.append_trace(go.Histogram2d(x=temp_data["x"]
                                       ,y=temp_data["div"]
                                       ,xbins={"start":0,"end":1,"size":1/n_buckets}
                                       ,ybins={"start":0,"end":divisions,"size":1}
                                       ,showlegend=show_legend[i]
                                       ,colorscale="ice")#[[0,colors[0],[0.5,colors[1]],[1,colors[2]]]])
                                    ,row=i+1,col=1)
   x = np.linspace(0.001,1,5000)
   y = [function([i]) for i in x]
   fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=n_plots+1,col=1)
   fig.update_layout(margin=dict(l=10, r=10, t=30, b=20),width=1000,height=800,plot_bgcolor='rgba(0,0,0,0)',title={"text":title}
                #,legend=dict(
                    #title = "Formula",
                    #orientation="h",
                    #yanchor="top",
                    #y=-0.65,
                    #xanchor="center",
                    #x=0.5,  
                    #font = dict(family = "Arial", size = 14, color = "black"),
                    #bordercolor="LightSteelBlue",
                    #borderwidth=2,
                    #itemsizing='trace',
                    #itemwidth = 30
                    #)  
                  )
   #fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black')
   fig.show()

def multiple_runs(state, function, player, runs, random_seed, divisions, dimension=0, division_method = "percentage", early_cut=False):
   random.seed(random_seed)
   random_seed_sequence = [random.randint(0,1000000) for _ in range(runs)]
   collective_data = []
   fo_logs = defaultdict(lambda:[])
   for run in range(runs):
      temp_player=player.ClonePlayer()
      random.seed(random_seed_sequence[run])
      np.random.seed(seed=random_seed_sequence[run])
      temp_player.chooseAction(state)
      data = tree_data(temp_player, divisions, dimension, early_cut)
      data.insert(0,"run",[run for _ in range(len(data))])
      collective_data.append(data)
      trt = False
      terminal_count = sum([1 for k,n in temp_player.nodes_dict.items() if n.untried_moves==[] and n.child ==[]])
      if terminal_count >= 1:
         trt = True
      fo_logs["Player"].append(temp_player.name)
      fo_logs["Tree_Nodes"].append(len(temp_player.nodes_dict))
      fo_logs["Max_Visits_Path"].append(function(best_tree_path(temp_player.nodes_dict[0],"visits").state.eval_point()))
      fo_logs["Max_Reward_Path"].append(function(best_tree_path(temp_player.nodes_dict[0]).state.eval_point()))
      fo_logs["Tree_Reaches_Terminal"].append(trt)
      fo_logs["Terminals_Reached"].append(terminal_count)
      fo_logs["Random_Seed"].append(random_seed_sequence[run])
   df_data = pd.concat(collective_data)
   fo_logs = pd.DataFrame(fo_logs)
   return fo_logs, df_data


for func_index, used_func in enumerate(function_list):
   for c_param in [0.5,1,math.sqrt(2),2,3]:
      #Experiment setup
      runs = 30
      random_seed = int(c_param*10)+func_index
      random.seed(random_seed)
      np.random.seed(seed=random_seed)
      logfile="FO_test2_"+str(func_index)+"_"+str(c_param)

      #Problem parameters
      branching_factor = 2
      #func_index = 1
      #used_func = paper_funcs[func_index]
      ranges = [[0,1]]

      #Player parameters
      iterations = 5000
      lamb = 4
      ngen = 20
      es_sims = 30
      #c_param = 1#math.sqrt(2)
      seml = 0.1
      semu = 0.5
      estype = "comma"

      #visualization parameters
      divisions = 3
      buckets = 1000
      buckets_2d = 10
      divisions_2d=100
      title = "c: " + "{:2.2f}".format(c_param) + ", bf:"+ str(branching_factor)+ ",it:" +str(iterations)

      #Players defnition
      mcts_player = MCTSPlayer(iterations=iterations
                              ,c_param = c_param
                              ,name='MCTS'
                              ,logs=True
                              ,logfile=logfile)
                              #,random_seed = random_seed)
      s_b_mcts_player = MCTS_ES_BACK_SEM_Player(iterations=iterations-(lamb*ngen*es_sims)
                              ,c_param=c_param
                              ,name='SE_MCTS'
                              ,Lambda=lamb
                              ,NGen=ngen
                              ,ES_Sims=es_sims
                              ,ESType=estype
                              ,logs=True
                              ,logfile=logfile
                              ,Sem_L=seml
                              ,Sem_U=semu)
      random_player = RandomPlayer()
      players = [mcts_player]#, s_b_mcts_player]

      #Execution
      state = FunctionOptimisationState(players=[mcts_player], function=used_func, ranges=ranges, splits=branching_factor)# give any single player
      used_func = state.function
      #state = FunctionOptimisationState(players=[random_player], function=func2, ranges=[[0,1],[0,1]], splits=3, minimum_step=0.0000001) #multiple dimensions

      all_data = []
      all_fo_logs = pd.DataFrame()
      for p_id, player in enumerate(players):
         fo_logs, data = multiple_runs(state, used_func, player, runs, random_seed, divisions)
         all_fo_logs = pd.concat([all_fo_logs,fo_logs])
         all_data.append(data)
         all_data_df = pd.concat(all_data)
         all_data_df.to_csv(os.path.join('logs', logfile, "Tree_data.csv"), index=False)
      #m2 = s_b_mcts_player.chooseAction(state)
      plot = show_search(all_data, used_func, title, n_buckets = buckets, divisions=divisions)
      plot.write_image(os.path.join('logs', logfile, "Averaged_plot" + '.png'), width=1920, height=1080)
      #show_2d_search([data], used_func, title, divisions_2d, n_buckets = buckets_2d)
      #show_search([mcts_player,s_b_mcts_player], used_func, title, n_buckets = 50, divisions=4)
      #print("Best reward path:",str(stats.mean(ors2)),str(stats.stdev(ors2)))



      #Create final logs
      #Combine separate logs
      exps.CombineFiles(logs=True,logfile=logfile)

      #Parameters logs
      final_logs={}
      final_logs["runs"]=runs
      final_logs["random_seed"]=random_seed
      final_logs["logfile"]=logfile
      #Problem parameters
      final_logs["branching_factor"]=branching_factor
      final_logs["func_index"]=func_index
      final_logs["ranges"]=ranges
      #Player parameters
      final_logs["iterations"]=iterations
      final_logs["lamb"]=lamb
      final_logs["ngen"]=ngen
      final_logs["es_sims"]=es_sims
      final_logs["c_param"]=c_param
      final_logs["seml"]=seml
      final_logs["semu"]=semu
      final_logs["estype"]=estype

      final_df = pd.DataFrame(final_logs)
      final_df.to_csv(os.path.join('logs', logfile, "Parameter_logs" + '.csv'), index=False)

      #Results logs
      player_logs = defaultdict(lambda:[])
      for player in players:
         temp_logs = all_fo_logs[all_fo_logs["Player"]==player.name]
         player_logs["Player"].append(player.name)
         player_logs["Mean_Tree_Nodes"].append(temp_logs["Tree_Nodes"].mean())
         player_logs["Std_Tree_Nodes"].append(temp_logs["Tree_Nodes"].std())
         player_logs["Mean_Terminals_Reached"].append(temp_logs["Terminals_Reached"].mean())
         player_logs["Std_Terminals_Reached"].append(temp_logs["Terminals_Reached"].std())
         player_logs["Mean_Max_Visits_Path"].append(temp_logs["Max_Visits_Path"].mean())
         player_logs["Std_Max_Visits_Path"].append(temp_logs["Max_Visits_Path"].std())
         player_logs["Mean_Max_Reward_Path"].append(temp_logs["Max_Reward_Path"].mean())
         player_logs["Std_Max_Reward_Path"].append(temp_logs["Max_Reward_Path"].std())
         player_logs["Tree_Reaches_Terminal"].append(temp_logs["Max_Reward_Path"].values.sum()/len(temp_logs))

      player_df = pd.DataFrame(player_logs)
      player_df.to_csv(os.path.join('logs', logfile, "Final_Player_logs" + '.csv'), index=False)
      all_fo_logs.to_csv(os.path.join('logs', logfile, "Player_logs" + '.csv'), index=False)




In [10]:
#### figure with functions


def func(x): 
   return 0.5 + 0.5*math.sin(20*x[0])
def func4(x): return abs(math.sin(1/math.pow(x[0]-0.5,1)))
def func5(x): #from the paper
   if x[0] < 0.5:
      return 0.5+0.5*abs(math.sin(1/pow(x[0],5)))
   else:
      return 7/20+0.5*abs(math.sin(1/pow(x[0],5)))
def func6(x): return 1/2*math.sin(13*x[0])*math.sin(27*x[0])+1
def func7(x): return 0.2*pow(math.sin(math.pi*5*x[0]),70)*1/(0.06+math.exp(x[0]-2))+pow(x[0],2)/2
def func8(x): return pow(math.sin(math.pi*3*x[0]),10)*0.18/(0.04+math.exp(x[0]-2))+pow(x[0],2)/2
def func9(x): return (-0.7*x[0]+1)*pow(math.sin(25*x[0]),60)+pow(x[0],2)/1.5
def func10(x): return (-0.6*x[0]+1)*pow(math.sin(math.pi*5*x[0]),60)+pow(x[0],2)/2
def func11(x): return (-0.6*x[0]+1)*pow(math.sin(31*x[0]),10)+pow(x[0],2)/2 #10 picos
def func12(x): 
   """10 peaks deceptive"""
   return (0.5*x[0])+(-0.7*x[0]+1)*pow(math.sin(5*math.pi*x[0]),4)
def func_p1(x): 
   """Unimodal, centered"""
   return (0.5*x[0])+(-0.7*x[0]+1)*pow(math.sin(5*math.pi*x[0]),80)
funcs = [func,func4,func5,func6,func7,func8,func9,func10,func12,func_p1]
def f0(x):
   """Unimodal, centered"""
   return math.sin(math.pi*x[0])
def f1(x):
   """Multimodal, paper bubeck"""
   return 1/2*(math.sin(13*x[0])*math.sin(27*x[0])+1)
def f2(x):
   """Smoothness with levels, paper finnsson"""
   if x[0] < 0.5:
      return 0.5+0.5*abs(math.sin(1/pow(x[0],5)))
   else:
      return 7/20+0.5*abs(math.sin(1/pow(x[0],5)))
def f3(x):
   """Deceptive"""
   return (0.5*x[0])+(-0.7*x[0]+1)*pow(math.sin(5*math.pi*x[0]),4)
def f4(x):
   """Deceptive, search traps"""
   return (0.5*x[0])+(-0.7*x[0]+1)*pow(math.sin(5*math.pi*x[0]),80)
function_list=[f0,f1,f2,f3,f4]

"""
x = np.linspace(0.001,1,5000)
for f in function_list:
   y = [f([i]) for i in x]
   fig = go.Figure(data=go.Scatter(x=x, y=y))
   fig.update_layout(title={"text":f.__name__})
   fig.show()
"""

title=""
n_plots = len(function_list)
even_spaces = 1/(n_plots)
row_heights = [even_spaces for _ in range(n_plots)]
sub_titles = ["Function " + str(i+1) for i in range(len(function_list))]
line_width = 1.5
fig = make_subplots(rows=n_plots, cols=1,shared_xaxes=True,vertical_spacing=0.05,row_heights=row_heights, subplot_titles=sub_titles)
   #,subplot_titles=("Function 1","Function 2","Function 3","Function 4","Function 5"))

show_legend = [False] + [False for _ in range(n_plots)]
max_x = [0.5,0.867,0,0.1,0.1]
for i,f in enumerate(function_list):
   x = np.linspace(0.001,1,5000)
   y = [f([i]) for i in x]
   fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=i+1,col=1)
   fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
   fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
   fig.update_xaxes(range=[0, 1])
   fig.update_yaxes(range=[0, 1])
   #fig.add_vline(x=max_x[i],line_dash="dot")
fig['layout'].update(shapes=[{'type': 'line','y0':0,'y1': 1,'x0':0.5, 
                              'x1':0.5,'xref':'x1','yref':'y1',
                              'line': {'color': "#B10909",'width': line_width, "dash":"dash"}},
                             {'type': 'line','y0':0,'y1': 1,'x0':0.867, 
                              'x1':0.867,'xref':'x2','yref':'y2',
                              'line': {'color': "#B10909",'width': line_width, "dash":"dash"}},
                              {'type': 'line','y0':0,'y1': 1,'x0':0.1, 
                              'x1':0.1,'xref':'x4','yref':'y4',
                              'line': {'color': "#B10909",'width': line_width, "dash":"dash"}},
                              {'type': 'line','y0':0,'y1': 1,'x0':0.1, 
                              'x1':0.1,'xref':'x5','yref':'y5',
                              'line': {'color': "#B10909",'width': line_width, "dash":"dash"}}])
fig.update_layout(margin=dict(l=10, r=10, t=10, b=20),width=800,height=800,plot_bgcolor='rgba(0,0,0,0)'
                  #,title={"text":"Functions of the Function Optimisation Problem"}
                  ,font = dict(family = "Arial", size = 14, color = "black")
               ,legend=dict(
                  #orientation="h",
                  #yanchor="top",
                  y=-0.65,
                  xanchor="center",
                  x=0.5,  
                  font = dict(family = "Arial", size = 14, color = "black"),
                  #bordercolor="LightSteelBlue",
                  borderwidth=2,
                  itemsizing='trace',
                  itemwidth = 30
                  )  )
#fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black')
fname_x = -0.08
fname_y_span = 0.225
fname_initial_y = 1
fname_size = 30
fname_names = ["f1","f2","f3","f4","f5"]
"""
for i,f_name in enumerate(fname_names):
   fig.add_annotation(
         x=fname_x,
         y=fname_initial_y-i*fname_y_span,
         xref="paper",
         yref="paper",
         text=" "+f_name+" ",
         showarrow=False,
         font=dict(
               #family="Courier New, monospace",
               size=fname_size,
               color="black"
               ),
         align="left",
         #arrowhead=2,
         #arrowsize=1,
         #arrowwidth=2,
         #arrowcolor="#636363",
         #ax=20,
         #ay=-30,
         bordercolor="black",
         borderwidth=2,
         #borderpad=4,
         #bgcolor="#ff7f0e",
         #opacity=0.8
         )
"""
fig.write_image(os.path.join('logs', "Functions" + '.png'))
fig.show()


In [ ]:
player1 = RandomPlayer()
player2 = RandomPlayer()

state = CarcassonneState(player1, player2, no_monasteries = True, no_farms = True)
while (not state.isGameOver):
    
    if state.playerSymbol == 1:
        m = player1.chooseAction(state)
    else:
        m = player2.chooseAction(state) # make move

    state.move(m)
print(state.winner)

(Random)   TimeTaken: 0 secs  -  Turn: 10  -  Time:04:16:24
(Random)   TimeTaken: 0 secs  -  Turn: 20  -  Time:04:16:24
(Random)   TimeTaken: 0 secs  -  Turn: 30  -  Time:04:16:24
(Random)   TimeTaken: 0 secs  -  Turn: 40  -  Time:04:16:24
(Random)   TimeTaken: 0 secs  -  Turn: 50  -  Time:04:16:24
(Random)   TimeTaken: 0 secs  -  Turn: 60  -  Time:04:16:24
2


In [ ]:
import os
import pandas as pd
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import numpy as np
import ast
import random
import math
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict

print(os.path.abspath(os.getcwd()))

logs_path = "logs"
exp_logs_names = ["Experiment_COMMA_WEAKS2022-01-07",
                    "Experiment_PLUS_WEAKS2022-01-03",
                    "Experiment_PLUS_WEAKS_REDUCED2022-01-10",
                    "Experiment_COMMA_WEAKS_REDUCED2022-01-11",
                    "Experiment_UL_SEM_C_2022-01-21",
                    "Experiment_UL_SEM_P_2022-01-20",
                    "4_EXP_All_Games"]

numbers = [str(x) for x in range(6)]
evostr_files = ["_ES_B_S_MCTS_EvoStr.csv", "_ES_B_MCTS_EvoStr.csv"]
join = "/"
uctstats = "UCTStats.csv"
semstats = "SemanticsStats.csv"

dfSSD = {}
uct_df = {}
holder = {}
sem_df = {}
for logs_name in exp_logs_names:
    dfSSD[logs_name] = pd.read_csv(logs_path + join + logs_name + join + "dfSSD.csv")
    uct_df[logs_name] = pd.read_csv(logs_path + join + logs_name + join + uctstats)
    sem_df[logs_name] = pd.read_csv(logs_path + join + logs_name + join + semstats)
    holder[logs_name] = {}
    for file in evostr_files:
        holder[logs_name][file] = {}
        for n in numbers:
            try:
                holder_filename = logs_path + join + logs_name + join + n + file
                holder[logs_name][file][n] = pd.read_csv(holder_filename)
            except:
                print(holder_filename, " was not found")
uct_df[exp_logs_names[0]].head()

d:\Users\fredx\Documents\Gits\mcts_uct_fred
logs/Experiment_PLUS_WEAKS_REDUCED2022-01-10/1_ES_B_S_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/0_ES_B_S_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/1_ES_B_S_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/2_ES_B_S_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/3_ES_B_S_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/4_ES_B_S_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/5_ES_B_S_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/0_ES_B_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/1_ES_B_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/2_ES_B_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/3_ES_B_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/4_ES_B_MCTS_EvoStr.csv  was not found
logs/Experiment_UL_SEM_C_2022-01-21/5_ES_B_MC

,Name,Turn,Generation,Lambda,TotalNodes,AverageNodes,AverageDepth,AverageSSD
0,ES_B_S_MCTS,1,1,4,52,10.4,4.4,5.75
1,ES_B_S_MCTS,1,2,4,78,15.6,6.6,7.20
2,ES_B_S_MCTS,1,3,4,66,13.2,4.6,7.55
3,ES_B_S_MCTS,1,4,4,64,12.8,3.6,4.70
4,ES_B_S_MCTS,1,5,4,77,15.4,3.4,9.05


In [ ]:
by_turn = {}
for exp in exp_logs_names:
    df = uct_df[exp]
    averages = []
    stdevs = []
    max_turn = int(max(df["Turn"].values.tolist()))
    turns = list(range(1,max_turn+1))
    for turn in turns:
        list_values = df.loc[df["Turn"]==turn,["AverageSSD"]].values.tolist()
        average = np.average(list_values)
        stdev = np.std(list_values)
        averages.append(average)
        stdevs.append(stdev)
    data = {"Turn":turns, "AverageSSD":averages, "StdevSSD":stdevs}
    by_turn[exp] = pd.DataFrame(data)

    #print(by_turn[exp].head())

In [ ]:
by_gen = {}
for exp in exp_logs_names:
    df = uct_df[exp]
    averages = []
    stdevs = []
    max_gen = int(max(df["Generation"].values.tolist()))
    gens = list(range(1,max_gen+1))
    for gen in gens:
        list_values = df.loc[df["Generation"]==gen,["AverageSSD"]].values.tolist()
        average = np.average(list_values)
        stdev = np.std(list_values)
        averages.append(average)
        stdevs.append(stdev)
    data = {"Generation":gens, "AverageSSD":averages, "StdevSSD":stdevs}
    by_gen[exp] = pd.DataFrame(data)

    #print(by_gen[exp].head())

In [ ]:
import matplotlib.pyplot as plt

SSD_by_turn = {}
for exp in exp_logs_names:
    fig, ax = plt.subplots(figsize=(10,5))
    SSD_by_turn[exp] = dfSSD[exp].boxplot(ax=ax, column = "SSD", by = "Turn", figsize=(10,5), return_type='axes')
    ax.set_ylim([0,20])


In [ ]:
uct_df_siea = {}
uct_df_ea = {}
for exp in exp_logs_names:
    uct_df_siea[exp] = uct_df[exp].loc[uct_df[exp]["Name"]=="ES_B_S_MCTS"]
    uct_df_ea[exp] = uct_df[exp].loc[uct_df[exp]["Name"]=="ES_B_MCTS"]

    fig, ax = plt.subplots(figsize=(10,5))
    SSD_by_gen = uct_df_siea[exp].boxplot(ax=ax, column = "AverageSSD", by = "Generation", figsize=(10,5), return_type='axes')
    ax.set_ylim([0,20])
    
    ax.set_title("SIEA_MCTS SSD by Generation : " + exp)

    fig, ax = plt.subplots(figsize=(10,5))
    SSD_by_gen = uct_df_ea[exp].boxplot(ax=ax, column = "AverageSSD", by = "Generation", figsize=(10,5), return_type='axes')
    ax.set_ylim([0,20])
    
    ax.set_title("EA_MCTS SSD by Generation : " +  exp)


In [ ]:
def check_ratio(L,U,return_as="Decisive_by_turn",experiments_with=None): 
    total = 0
    decisive_count = 0
    min_count = 0
    decisive_by_turn = defaultdict(lambda: 0)
    min_by_turn = defaultdict(lambda: 0)
    not_random_by_turn = defaultdict(lambda: 0)
    total_by_turn = defaultdict(lambda: 0)
    relevant_ssds_turn = defaultdict(lambda: [])
    ssds = []

    if experiments_with is not None:
        exps = [exp for exp in exp_logs_names if experiments_with in exp]
    else:
        exps = exp_logs_names
        
    for exp in exps:
        df = sem_df[exp]
        for index, row in df.iterrows():
            fitnesses = row.Fitnesses
            semantics = row.SSDs
            fitnesses = fitnesses.replace(",","")
            fitnesses = fitnesses.replace("(","")
            fitnesses = fitnesses.replace("[","")
            semantics = semantics.replace("[","")
            fitnesses = fitnesses.replace("]","")
            semantics = semantics.replace("]","")
            fitnesses = fitnesses.replace(",","")
            fitnesses = fitnesses[:-1]
            fitnesses = fitnesses.split(")")
            semantics = semantics.split(",")
            fitnesses = [float(f) for f in fitnesses]
            semantics = [float(f) for f in semantics]
            max_fitness = max(fitnesses[1:])
            max_indexes = [i for i,v in enumerate(fitnesses) if float(v) == max_fitness]
            filtered_semantics = [s for i,s in enumerate(semantics) if i in max_indexes]
            relevant_ssds_turn[row.Turn] += filtered_semantics
            in_range = [s for s in filtered_semantics if s < U and s > L]
            ssds += in_range
            if len(in_range) == 1: 
                decisive_count += 1
                decisive_by_turn[row.Turn] += 1
                not_random_by_turn[row.Turn] += 1
            if len(in_range) > 1: 
                min_count += 1
                min_by_turn[row.Turn] += 1
                not_random_by_turn[row.Turn] += 1
            total += 1
            total_by_turn[row.Turn] += 1


    """ Info zone
    print("stdev",stats.stdev(ssds))
    print("mean",stats.mean(ssds))
    print("quantiles",np.quantile(ssds, q = np.arange(0.25, 1, 0.25))) 
    fig = plt.figure(figsize =(10, 7))
    plt.boxplot(ssds)
    plt.show()
    """
    if return_as == "Decisive_by_turn":
        turn_ratios = []
        for turn, value in total_by_turn.items():
            turn_ratios.append(decisive_by_turn[turn]/value)   
        return stats.mean(turn_ratios)

    if return_as == "Decisive":
        return decisive_count/total

    if return_as == "Min":
        return min_count/total

In [ ]:
print(check_ratio(0,50))

0.4471804646285321


In [ ]:
step_size = 1
min_step_size = 0.001

L = 0
U = 20
best_L = L
best_U = U
best_ratio = 0
L_turn = True

while step_size > min_step_size:
    
    ratio = check_ratio(L,U, experiments_with="All")
    #ratio = check_ratio(L,U, "Decisive")
    if ratio >= best_ratio: 
        best_ratio = ratio
        best_L = L
        best_U = U
        print("New best! L: ",L, " U: ", U, " Ratio: ", ratio)
    
        if L_turn:
            L += step_size
        else:
            U -= step_size
    
    
    else:
        if L_turn:
            L -= step_size
            L_turn = False
        else:
            U += step_size
            L_turn = True
            step_size = step_size/10

    

New best! L:  0  U:  20  Ratio:  0.7794219447424455
New best! L:  0  U:  20  Ratio:  0.7794219447424455
New best! L:  0  U:  19  Ratio:  0.7794219447424455
New best! L:  0  U:  18  Ratio:  0.7794219447424455
New best! L:  0  U:  17  Ratio:  0.7794219447424455
New best! L:  0  U:  16  Ratio:  0.7794219447424455
New best! L:  0  U:  15  Ratio:  0.7794219447424455
New best! L:  0  U:  14  Ratio:  0.7794219447424455
New best! L:  0  U:  13  Ratio:  0.7794219447424455
New best! L:  0  U:  12  Ratio:  0.7794219447424455
New best! L:  0  U:  11  Ratio:  0.7794219447424455
New best! L:  0  U:  10  Ratio:  0.7794219447424455
New best! L:  0  U:  9  Ratio:  0.7794219447424455
New best! L:  0  U:  8  Ratio:  0.7794219447424455
New best! L:  0  U:  7  Ratio:  0.7799666070518138
New best! L:  0  U:  7  Ratio:  0.7799666070518138
New best! L:  0.0  U:  7  Ratio:  0.7799666070518138
New best! L:  0.0  U:  7.0  Ratio:  0.7799666070518138
New best! L:  0.01  U:  7.0  Ratio:  0.7799666070518138
New best

In [ ]:
import pandas as pd
import os
import numpy as np
import math
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"
import plotly.express as px
from plotly.subplots import make_subplots
  
#UCTStats:
#Name	Turn	Generation	Lambda	TotalNodes	AverageNodes	AverageDepth	AverageSSD	IsFirstPlayer	Opponent

#Plot options:
#https://plotly.com/python/creating-and-updating-figures/
#https://pythonwife.com/bar-charts-with-plotly/

print(os.path.abspath(os.getcwd()))

logs_path = "logs"
join = "/"
uctstats = "UCTStats.csv"
semstats = "SemanticsStats.csv"
prom_exps = ["Experiment_UL_SEM_C_2022-01-30",
            "Experiment_UL_SEM_P_2022-01-20"]
prev_exps =  ["Experiment_S2_WEAKS2022-01-31",
            "Experiment_PLUS_WEAKS2022-01-03",
            "Experiment_PLUS_WEAKS_REDUCED2022-01-10",
            "Experiment_COMMA_WEAKS_REDUCED2022-01-11"]
L = 0.39
U = 8.34



def get_players(exp_name):
    league_table = pd.read_csv(logs_path + join + exp_name + join + "FinalLeagueTable.csv")
    players = league_table["Player"].to_list()
    return players

def get_range(df_fitnesses, df_semantics, L, U, comma=False):
    ranges = []
    for index, fitnesses in enumerate(df_fitnesses):
        semantics = df_semantics.iloc[index]
        fitnesses = fitnesses.replace(",","")
        fitnesses = fitnesses.replace("(","")
        fitnesses = fitnesses.replace("[","")
        semantics = semantics.replace("[","")
        fitnesses = fitnesses.replace("]","")
        semantics = semantics.replace("]","")
        fitnesses = fitnesses.replace(",","")
        fitnesses = fitnesses[:-1]
        fitnesses = fitnesses.split(")")
        semantics = semantics.split(",")
        fitnesses = [float(f) for f in fitnesses]
        semantics = [float(f) for f in semantics]
        max_fitness = max(fitnesses[1:])
        if comma:
            max_indexes = [i for i,v in enumerate(fitnesses) if float(v) == max_fitness and i!=0]
        else:
            max_indexes = [i for i,v in enumerate(fitnesses) if float(v) == max_fitness]
        filtered_semantics = [s for i,s in enumerate(semantics) if i in max_indexes]
        in_range = [s for s in filtered_semantics if s < U and s > L]
        ranges.append(len(in_range))
    return ranges

def define_tiebreak(fitnesses, semantics, L, U, comma=False):
    tiebreaks = []
    ranges = get_range(fitnesses, semantics, L, U, comma)
    #print(ranges)
    for n_in_range in ranges:
        if n_in_range == 1: tiebreaks.append("Decisive")
        elif n_in_range >= 2: tiebreaks.append("Minimum")
        else: tiebreaks.append("Random")
    return tiebreaks

def semantics_stats_analysis(exp_name, L, U, plot_random_by_turn=True, plot_ssd_by_turn=True, plot_equals_by_turn=True, comma=False):
    all = {}

    #get needed variables
    games_data = pd.read_csv(logs_path + join + exp_name + join + "GamesCompleted.csv")
    n_games = games_data.Total_Games[0]
    
    #Semantics stats
    stats = pd.read_csv(logs_path + join + exp_name + join + "SemanticsStats.csv")
    equal_fitnesses = len(stats)

    #Semantics stats
    uct_stats = pd.read_csv(logs_path + join + exp_name + join + "UCTStats.csv")

    #Formulas stats
    formulas = pd.read_csv(logs_path + join + exp_name + join + "EvolvedUCT.csv")

    #Calculate decisives and minimums
    stats = stats.assign(tiebreak=lambda x: define_tiebreak(x.Fitnesses, x.SSDs, L, U, comma))
    stats = stats.assign(decisive=lambda x: x.tiebreak=="Decisive")
    stats = stats.assign(minimum=lambda x: x.tiebreak=="Minimum")

    #Grouped by turn
    uct_stats_by_turn = uct_stats.groupby(["Turn"]).mean()
    stats_by_turn = stats.groupby(["Turn"]).mean()

    #assignation
    all["n_games"] = n_games
    all["equal_fitnesses"] = equal_fitnesses
    all["equal_fitnesses_by_game"] = equal_fitnesses/n_games
    all["randoms"] = stats.WasRandom.mean()
    all["stats"] = stats
    all["uct_stats_by_turn"] = uct_stats_by_turn
    all["stats_by_turn"] = stats_by_turn
    all["formulas"] = formulas

    return all





d:\Users\fredx\Documents\Gits\mcts_uct_fred


In [ ]:
#### Many bar plots!!


sem_stats = semantics_stats_analysis(prom_exps[0],L,U,comma=True)
sem_stats_p = semantics_stats_analysis(prom_exps[1],L,U)

old_sem_stats = semantics_stats_analysis(prev_exps[0],L=5,U=10, comma=True)
old_sem_stats_p = semantics_stats_analysis(prev_exps[1],L=5,U=10)


###########Tiebreak distribution
exps_data = [sem_stats["stats"], sem_stats_p["stats"], old_sem_stats["stats"], old_sem_stats_p["stats"]]
exps_names = ["Comma_PS", "Plus_PS", "Comma", "Plus"]
tiebreaks = ["Random","Decisive","Minimum"]
colors = {'Random': 'red',
          'Minimum': 'orange',
          'Decisive': 'lightgreen'}



plot = make_subplots(rows=2, cols=3
    , shared_yaxes=True
    , subplot_titles=("Full experiment", "Without turn 36", "With turns up to 33")
    , x_title="Data included"
)
legend_not_added = True
for t in tiebreaks:
    y = [len(df[df["tiebreak"]==t])/len(df) for df in exps_data]
    plot.add_trace(go.Bar(
        name = t,
        x = exps_names,
        y=y,
        showlegend=legend_not_added
        ,marker={'color': colors[t]}
        )
    ,row=1
    ,col=1
    )

    y = [len(df[df["tiebreak"]==t][df["Turn"]!=36])/len(df[df["Turn"]!=36]) for df in exps_data]
    plot.add_trace(go.Bar(
        name = t,
        x = exps_names,
        y=y,
        showlegend=False
        ,marker={'color': colors[t]}
        )
    ,row=1
    ,col=2
    )

    y = [len(df[df["tiebreak"]==t][df["Turn"]<34])/len(df[df["Turn"]<34]) for df in exps_data]
    plot.add_trace(go.Bar(
        name = t,
        x = exps_names,
        y=y,
        showlegend=False
        ,marker={'color': colors[t]}
        )
    ,row=1
    ,col=3
    )
    
for t in tiebreaks:
    y = [len(df[df["tiebreak"]==t]) for df in exps_data]
    plot.add_trace(go.Bar(
        name = t,
        x = exps_names,
        y=y,
        showlegend=legend_not_added
        ,marker={'color': colors[t]}
        )
    ,row=2
    ,col=1
    )

    y = [len(df[df["tiebreak"]==t][df["Turn"]!=36]) for df in exps_data]
    plot.add_trace(go.Bar(
        name = t,
        x = exps_names,
        y=y,
        showlegend=False
        ,marker={'color': colors[t]}
        )
    ,row=2
    ,col=2
    )

    y = [len(df[df["tiebreak"]==t][df["Turn"]<34]) for df in exps_data]
    plot.add_trace(go.Bar(
        name = t,
        x = exps_names,
        y=y,
        showlegend=False
        ,marker={'color': colors[t]}
        )
    ,row=2
    ,col=3
    )
    
plot.update_layout(barmode='stack'
            ,title="Tiebreak count and distribution"
            #,xaxis_title="Experiment"
            #,yaxis_title="Count"
            ,legend_title="Tiebreak")
plot["layout"]["yaxis1"]["title"] = "Distribution"
plot["layout"]["yaxis4"]["title"] = "Count"
#plot.update_xaxes(showgrid=True, gridwidth=1)        
plot.show()




###########Tiebreak count
plot = go.Figure()
for t in tiebreaks:
    y = [len(df[df["tiebreak"]==t]) for df in exps_data]
    plot.add_trace(go.Bar(
        name = t,
        x = exps_names,
        y=y
    ))
plot.update_layout(barmode='stack'
            ,title="Tiebreak total if equal fitness"
            ,xaxis_title="Experiment"
            ,yaxis_title="Amount"
            ,legend_title="Tiebreak")              
plot.show()





###########Tiebreak by turn
legend_not_added = True
colors = {'Random': 'red',
          'Minimum': 'orange',
          'Decisive': 'lightgreen'}
plot = make_subplots(rows=4, cols=1
    , shared_xaxes=True
    , subplot_titles=(exps_names)
    , row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for row, full_df in enumerate(exps_data):
    x = full_df["Turn"].unique()
    for t in tiebreaks:
        y=[]
        for turn in x:
            df = full_df[full_df["Turn"]==turn]
            y.append(len(df[df["tiebreak"]==t])/len(df))
        plot.add_trace(go.Bar(
                name=t,x=x,y=y
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                ,showlegend=legend_not_added
                ),
            row = row+1,
            col = 1)
    if legend_not_added:
        legend_not_added = False
plot.update_layout(barmode='stack'
            ,title_text="Tiebreak distribution by turn"
            ,xaxis_title="Turn"
            ,yaxis_title="Proportion"
            ,legend_title="Tiebreak"
            ,autosize=False
            ,width=1000
            ,height=800)                
plot.show()


legend_not_added = True
colors = {'Random': 'red',
          'Minimum': 'orange',
          'Decisive': 'lightgreen'}
plot = make_subplots(rows=4, cols=1
    , shared_xaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for row, full_df in enumerate(exps_data):
    x = full_df["Turn"].unique()
    for t in tiebreaks:
        y=[]
        for turn in x:
            df = full_df[full_df["Turn"]==turn]
            y.append(len(df[df["tiebreak"]==t]))
        plot.add_trace(go.Bar(
                name=t,x=x,y=y
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                ,showlegend=legend_not_added
                ),
            row = row+1,
            col = 1)
    if legend_not_added:
        legend_not_added = False
plot.update_layout(barmode='stack'
            ,title_text="Tiebreak count by turn"
            ,xaxis_title="Turn"
            ,yaxis_title="Count"
            ,legend_title="Tiebreak"
            ,autosize=False
            ,width=1000
            ,height=800)                
plot.show()

legend_not_added = True
colors = {'Random': 'red',
          'Minimum': 'orange',
          'Decisive': 'lightgreen'}
plot = make_subplots(rows=4, cols=1
    , shared_xaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for row, full_df in enumerate(exps_data):
    x = full_df["Turn"].unique()
    for t in tiebreaks:
        y=[]
        for turn in x:
            df = full_df[full_df["Turn"]==turn]
            y.append(len(df[df["tiebreak"]==t]))
        plot.add_trace(go.Bar(
                name=t,x=x,y=y
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                ,showlegend=legend_not_added
                ),
            row = row+1,
            col = 1)
    if legend_not_added:
        legend_not_added = False
plot.update_layout(barmode='stack'
            ,title_text="Tiebreak count by turn"
            ,xaxis_title="Turn"
            ,yaxis_title="Count"
            ,legend_title="Tiebreak"
            ,autosize=False
            ,width=1000
            ,height=800)                
plot.show()





colors = {'Random': 'red',
          'Minimum': 'orange',
          'Decisive': 'lightgreen'}
plot = make_subplots(rows=4, cols=1
    , shared_xaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for row, full_df in enumerate(exps_data):
     x = full_df["Generation"].unique()
     for t in tiebreaks:
        y=[]
        for gen in x:
            df = full_df[full_df["Generation"]==gen][full_df["Turn"]<=34]
            y.append(len(df[df["tiebreak"]==t]))
        plot.add_trace(go.Bar(
                name=t,x=x,y=y
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                #showlegend=False),
                ),
            row = row+1,
            col = 1)
plot.update_layout(barmode='stack'
            ,title_text="Tiebreak count by generation with turns up to 33"
            ,xaxis_title="Generation"
            ,yaxis_title="Count"
            ,legend_title="Tiebreak"
            ,autosize=False
            ,width=1000
            ,height=800)                
plot.show()

#missing: correlation between average nodes and ssd by generation


In [ ]:
colors = {'Random': 'red',
          'Minimum': 'orange',
          'Decisive': 'lightgreen'}

sem_stats = semantics_stats_analysis(prom_exps[0],L,U,comma=True)
sem_stats_p = semantics_stats_analysis(prom_exps[1],L,U)
#old_sem_stats = semantics_stats_analysis(prev_exps[0],L,U, comma=True)
#old_sem_stats_p = semantics_stats_analysis(prev_exps[1],L,U)

exps_data = [sem_stats["stats"], sem_stats_p["stats"]]#, old_sem_stats["stats"], old_sem_stats_p["stats"]]
exps_names = ["Comma_PS", "Plus_PS"]#, "Comma", "Plus"]
tiebreaks = ["Random","Decisive","Minimum"]


plot = make_subplots(rows=2, cols=2
    , shared_yaxes=True
    , subplot_titles=(exps_names + [n + " excluding turn 36" for n in exps_names])
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True
    , x_title="Is first player?"
    , y_title="Count"
    )
leyend_needed = True

for col, full_df in enumerate(exps_data):

    x = full_df["IsFirstPlayer"].unique()
    for t in tiebreaks:
        y=[]
        y2 = []
        for p in x:
            df = full_df[full_df["IsFirstPlayer"]==p]
            df2 = full_df[full_df["IsFirstPlayer"]==p][full_df["Turn"]!=36]
            #df = full_df.loc[((full_df['IsFirstPlayer'] == p) & (full_df['Turn'] == 36))]
            y.append(len(df[df["tiebreak"]==t]))
            y2.append(len(df2[df2["tiebreak"]==t]))
        plot.add_trace(go.Bar(
                name=t,x=x,y=y
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                ,showlegend=leyend_needed
                ),
            row = 1,
            col = 1+col)
        plot.add_trace(go.Bar(
                name=t,x=x,y=y2
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                ,showlegend=False
                ),
            row = 2,
            col = 1+col)
    leyend_needed = False
plot.update_layout(barmode='stack'
            ,title_text="Tiebreak count when first or second player"
            #,xaxis_title="Is first player?"
            #,yaxis_title="Count"
            ,legend_title="Tiebreak"
            ,autosize=False
            ,width=1000
            ,height=400)  
#plot['layout']['xaxis2']['title']="Is first player?"                
plot.show()
##Note: first player gets an extra turn



plot = make_subplots(rows=1, cols=2
    , shared_yaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for col, full_df in enumerate(exps_data):
     x = full_df["IsFirstPlayer"].unique()
     for t in tiebreaks:
        y=[]
        for p in x:
            df = full_df[full_df["IsFirstPlayer"]==p]
            y.append(len(df[df["tiebreak"]==t])/len(df))
        plot.add_trace(go.Bar(
                name=t,x=x,y=y
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                #showlegend=False),
                ),
            row = 1,
            col = 1+col)
plot.update_layout(barmode='stack'
            ,title_text="Tiebreak distribution when first or second player"
            ,xaxis_title="Is first player?"
            ,yaxis_title="Proportion"
            ,legend_title="Tiebreak"
            ,autosize=False
            ,width=1000
            ,height=400)                
plot.show()

In [ ]:
colors = {'Random': 'red',
          'Minimum': 'orange',
          'Decisive': 'lightgreen'}

sem_stats = semantics_stats_analysis(prom_exps[0],L,U,comma=True)
sem_stats_p = semantics_stats_analysis(prom_exps[1],L,U)
#old_sem_stats = semantics_stats_analysis(prev_exps[0],L,U, comma=True)
#old_sem_stats_p = semantics_stats_analysis(prev_exps[1],L,U)

exps_data = [sem_stats["stats"], sem_stats_p["stats"]]#, old_sem_stats["stats"], old_sem_stats_p["stats"]]
exps_names = ["Comma_PS", "Plus_PS"]#, "Comma", "Plus"]
tiebreaks = ["Random","Decisive","Minimum"]


plot = make_subplots(rows=1, cols=2
    , shared_yaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for col, full_df in enumerate(exps_data):
     x = full_df["Opponent"].unique()
     for t in tiebreaks:
        y=[]
        for p in x:
            df = full_df[full_df["Opponent"]==p]
            y.append(len(df[df["tiebreak"]==t]))
        plot.add_trace(go.Bar(
                name=t,x=x,y=y
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                #showlegend=False),
                ),
            row = 1,
            col = 1+col)
plot.update_layout(barmode='stack'
            ,title_text="Tiebreak count by opponent"
            ,xaxis_title="Opponent"
            ,yaxis_title="Count"
            ,legend_title="Tiebreak"
            ,autosize=False
            ,width=1000
            ,height=400)                
plot.show()
##Note: first player gets an extra turn

plot = make_subplots(rows=1, cols=2
    , shared_yaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for col, full_df in enumerate(exps_data):
     x = full_df["Opponent"].unique()
     for t in tiebreaks:
        y=[]
        for p in x:
            df = full_df[full_df["Opponent"]==p]
            y.append(len(df[df["tiebreak"]==t])/len(df))
        plot.add_trace(go.Bar(
                name=t,x=x,y=y
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                #showlegend=False),
                ),
            row = 1,
            col = 1+col)
plot.update_layout(barmode='stack'
            ,title_text="Tiebreak distribution by opponent"
            ,xaxis_title="Opponent"
            ,yaxis_title="Proportion"
            ,legend_title="Tiebreak"
            ,autosize=False
            ,width=1000
            ,height=400)                
plot.show()

In [ ]:

sem_stats = semantics_stats_analysis(prom_exps[0],L,U,comma=True)
sem_stats_p = semantics_stats_analysis(prom_exps[1],L,U)
#old_sem_stats = semantics_stats_analysis(prev_exps[0],L,U, comma=True)
#old_sem_stats_p = semantics_stats_analysis(prev_exps[1],L,U)

exps_data = [sem_stats["stats"], sem_stats_p["stats"]]#, old_sem_stats["stats"], old_sem_stats_p["stats"]]
exps_names = ["Comma_PS", "Plus_PS"]#, "Comma", "Plus"]
tiebreaks = ["Random","Decisive","Minimum"]

full_df = exps_data[1]

plot = go.Figure()
x = full_df["Generation"].unique()
y=[]
for p in x:
    df = full_df[full_df["Generation"]==p]
    y.append(len(df[df["BestIndex"]==0])/len(df))
plot.add_trace(go.Bar(
        name=t,x=x,y=y
        ))
plot.update_layout(barmode='stack'
            ,title_text="Plus: Rate of parent kept as winner"
            ,xaxis_title="Generation"
            ,yaxis_title="Rate"
            ,autosize=False
            ,width=500
            ,height=400)                
plot.show()

In [ ]:
##Formulas

import statistics as stats

sem_stats = semantics_stats_analysis(prom_exps[0],L,U,comma=True)
sem_stats_p = semantics_stats_analysis(prom_exps[1],L,U)
old_sem_stats = semantics_stats_analysis(prev_exps[0],5,10, comma=True)
old_sem_stats_p = semantics_stats_analysis(prev_exps[1],5,10 )

exps_data = [sem_stats["formulas"], sem_stats_p["formulas"], old_sem_stats["formulas"], old_sem_stats_p["formulas"]]
exps_names = ["Comma_PS", "Plus_PS", "Comma", "Plus"]

terminals = ["Q","N","n"]

legend_not_added = True
colors = {'Q': 'red',
          'N': 'orange',
          'n': 'lightgreen'}
plot = make_subplots(rows=4, cols=1
    , shared_xaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for row, full_df in enumerate(exps_data):
    x = full_df["Turn"].unique()
    for t in terminals:
        y=[]
        for turn in x:
            df = full_df[full_df["Turn"]==turn]
            #print("test len",df["Function"])
            y.append(len(df[df["Function"].str.contains(t)]))
        plot.add_trace(go.Scatter(
                name=t,x=x,y=y
                ,legendgroup='group1'
                ,marker={'color': colors[t]}
                ,showlegend=legend_not_added
                ,mode='lines+markers'
                ),
            row = row+1,
            col = 1)
    if legend_not_added:
        legend_not_added = False
plot.update_layout(#barmode='group'
            #,
            title_text="Terminal appearence count by turn"
            ,xaxis_title="Turn"
            ,yaxis_title="Count"
            ,legend_title="Terminal"
            ,autosize=False
            ,width=1000
            ,height=800)                
plot.show()



legend_not_added = True
plot = make_subplots(rows=4, cols=1
    , shared_xaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for row, full_df in enumerate(exps_data):
    full_df = full_df[full_df["IsDifferent"]]
    x = full_df["Turn"]
    y = full_df["NumberNodes"]
    plot.add_trace(go.Box(
        name="Number of Nodes",x=x,y=y
        ,legendgroup='group1'
        ,showlegend=legend_not_added
        ),
    row = row+1,
    col = 1)
    plot.add_trace(go.Scatter(x=x, y=[13 for _ in x],
                    mode='lines',legendgroup='group1'
                    ,showlegend=legend_not_added,
                    name='original UCT'),
    row = row+1,
    col = 1)
    if legend_not_added:
        legend_not_added = False
plot.update_layout(title_text="Number of nodes in the formula by turn (if different from original)"
            ,xaxis_title="Turn"
            ,yaxis_title="Value"
            ,autosize=False
            ,width=1000
            ,height=800)                
plot.show()

legend_not_added = True
plot = make_subplots(rows=4, cols=1
    , shared_xaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for row, full_df in enumerate(exps_data):
    full_df = full_df[full_df["IsDifferent"]]
    x = full_df["Turn"]
    y = full_df["Depth"]
    plot.add_trace(go.Box(
        name="Depth",x=x,y=y
        ,legendgroup='group1'
        ,showlegend=legend_not_added
        ),
    row = row+1,
    col = 1)
    plot.add_trace(go.Scatter(x=x, y=[6 for _ in x],
                    mode='lines',legendgroup='group1'
                    ,showlegend=legend_not_added,
                    name='original UCT'),
    row = row+1,
    col = 1)
    if legend_not_added:
        legend_not_added = False
plot.update_layout(title_text="Depth of the formula tree by turn (if different from original)"
            ,xaxis_title="Turn"
            ,yaxis_title="Value"
            ,autosize=False
            ,width=1000
            ,height=800)                
plot.show()


legend_not_added = True
plot = make_subplots(rows=4, cols=1
    , shared_xaxes=True
    , subplot_titles=(exps_names)
    #, row_heights=[4 for _ in exps_names]
    #, vertical_spacing=0.3
    , print_grid=True)
for row, full_df in enumerate(exps_data):
    x = full_df["Turn"].unique()
    y=[]
    strings = []
    for turn in x:
        df = full_df[full_df["Turn"]==turn]
        ratio = stats.mean(df["IsDifferent"])
        y.append(ratio)
        strings.append(str(ratio))
    plot.add_trace(go.Scatter(
            name=t,x=x,y=y,text=strings
            ,legendgroup='group1'
            ,showlegend=legend_not_added
            ,mode='lines+markers'
            ),
        row = row+1,
        col = 1)
    if legend_not_added:
        legend_not_added = False
plot.update_layout(
            #barmode='group'
            #,
            title_text="Formula changed by turn"
            ,xaxis_title="Turn"
            ,yaxis_title="Ratio"
            ,autosize=False
            ,width=1000
            ,height=800)                
plot.show()

In [ ]:
import pandas as pd
import os
import numpy as np
import math
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import ast
import random
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy as sp

print(os.path.abspath(os.getcwd()))

logs_path = "logs"
join = "/"

def scores_analysis(exp_name, join=join, logs_path=logs_path, plot_show=[True, True, True]):

    scores_data = pd.read_csv(logs_path + join + exp_name + join + "PlayerStats.csv")
    agents = scores_data["Player"].unique()
    n_agents = len(agents)
    print(agents)
    all_colors = ["blue","orange","green","yellow","red","black","pink","brown","purple","gray" ]
    assert(n_agents <= len(all_colors))
    colors = {opponent:all_colors[i] for i,opponent in enumerate(agents)}

    for agent in agents:
        featured = []
        for i, row in scores_data.iterrows():
            featured.append(row["Opponent"]==agent or row["Player"]==agent)
        scores_data["featured_"+agent] = featured

    ############ Score distribution as first

    if plot_show[0]==True:
        plot = make_subplots(rows=3, cols=1
            #,shared_xaxes=True
            #,shared_yaxes=True
            ,subplot_titles=["Score by opponent","Score difference by opponent","Score proportion by opponent"]
            #,row_heights=[4 for _ in exps_names]
            #,vertical_spacing=0.3
            ,print_grid=True
            ,x_title="Player")

        for i,opponent in enumerate(agents):
            
            filtered_data = scores_data[scores_data["Opponent"]==opponent]
            plot.add_trace(
                go.Box(
                    y=filtered_data["PlayerScore"],
                    x=filtered_data["Player"],
                    name=opponent,
                    marker_color=colors[opponent],
                    showlegend=True),
                row = 1,
                col = 1)

            plot.add_trace(
                go.Box(
                    y=filtered_data["PlayerScore"]-filtered_data["OpponentScore"],
                    x=filtered_data["Player"],
                    name=opponent,
                    marker_color=colors[opponent],
                    showlegend=False),
                row = 2,
                col = 1)

            plot.add_trace(
                go.Box(
                    y=filtered_data["PlayerScore"]/filtered_data["OpponentScore"],
                    x=filtered_data["Player"],
                    name=opponent,
                    marker_color=colors[opponent],
                    showlegend=False),
                row = 3,
                col = 1)

        plot.update_layout(
            boxmode='group'
            ,title_text="Score distribution as first player"
            #,xaxis_title="Player"
            ,yaxis_title="Score"
            ,legend_title="Opponent"
            ,autosize=False
            ,width=1300
            ,height=1200)
            
        plot.show()

    ############ Score distribution as second

    if plot_show[1]==True:
        plot = make_subplots(rows=3, cols=1
            #,shared_xaxes=True
            #,shared_yaxes=True
            ,subplot_titles=["Score by opponent","Score difference by opponent","Score proportion by opponent"]
            #,row_heights=[4 for _ in exps_names]
            #,vertical_spacing=0.3
            ,print_grid=True
            ,x_title="Player")

        for i,opponent in enumerate(agents):
            
            filtered_data = scores_data[scores_data["Opponent"]==opponent]
            plot.add_trace(
                go.Box(
                    y=filtered_data["PlayerScore"],
                    x=filtered_data["Player"],
                    name=opponent,
                    marker_color=colors[opponent],
                    showlegend=True),
                row = 1,
                col = 1)

            plot.add_trace(
                go.Box(
                    y=filtered_data["PlayerScore"]-filtered_data["OpponentScore"],
                    x=filtered_data["Player"],
                    name=opponent,
                    marker_color=colors[opponent],
                    showlegend=False),
                row = 2,
                col = 1)

            plot.add_trace(
                go.Box(
                    y=filtered_data["PlayerScore"]/filtered_data["OpponentScore"],
                    x=filtered_data["Player"],
                    name=opponent,
                    marker_color=colors[opponent],
                    showlegend=False),
                row = 3,
                col = 1)

        plot.update_layout(
            boxmode='group'
            ,title_text="Score distribution as second player"
            #,xaxis_title="Player"
            ,yaxis_title="Score"
            ,legend_title="Opponent"
            ,autosize=False
            ,width=1300
            ,height=1200)
            
        plot.show()

    ####### Opponent's scores

    if plot_show[2]==True:
        plot = make_subplots(rows=3, cols=2
            #,shared_xaxes=True
            #,shared_yaxes=True
            ,subplot_titles=["Opponent's combined score","Opponent's combined score difference","Opponent's combined score proportion"]
            #,row_heights=[4 for _ in exps_names]
            #,vertical_spacing=0.3
            ,print_grid=True
            ,x_title="Player")
        

        plot.add_trace(
                go.Box(
                    y=scores_data["OpponentScore"],
                    x=scores_data["Player"],
                    name="As 1st",
                    legendgroup='group1',
                    #marker_color=colors[opponent],
                    showlegend=True),
                row = 1,
                col = 1)

        plot.add_trace(
                go.Box(
                    y=scores_data["PlayerScore"]-scores_data["OpponentScore"],
                    x=scores_data["Player"],
                    legendgroup='group1',
                    #name=opponent,
                    #marker_color=colors[opponent],
                    showlegend=False),
                row = 2,
                col = 1)

        plot.add_trace(
                go.Box(
                    y=scores_data["PlayerScore"]/scores_data["OpponentScore"],
                    x=scores_data["Player"],
                    legendgroup='group1',
                    #name=opponent,
                    #marker_color=colors[opponent],
                    showlegend=False),
                row = 3,
                col = 1)

        plot.add_trace(
                go.Box(
                    y=scores_data["PlayerScore"],
                    x=scores_data["Opponent"],
                    name="As 2nd",
                    legendgroup='group2',
                    #marker_color=colors[opponent],
                    showlegend=True),
                row = 1,
                col = 2)

        plot.add_trace(
                go.Box(
                    y=scores_data["OpponentScore"]-scores_data["PlayerScore"],
                    x=scores_data["Opponent"],
                    legendgroup='group2',
                    #name=opponent,
                    #marker_color=colors[opponent],
                    showlegend=False),
                row = 2,
                col = 2)

        plot.add_trace(
                go.Box(
                    y=scores_data["OpponentScore"]/scores_data["PlayerScore"],
                    x=scores_data["Opponent"],
                    legendgroup='group2',
                    #name=opponent,
                    #marker_color=colors[opponent],
                    showlegend=False),
                row = 3,
                col = 2)

        plot.update_layout(
            boxmode='group'
            ,title_text="Score distribution"
            #,xaxis_title="Player"
            ,yaxis_title="Score"
            ,legend_title="Player"
            ,autosize=False
            ,width=1300
            ,height=800)
            
        plot.show()

    return scores_data, agents


d:\Users\fredx\Documents\Gits\mcts_uct_fred


In [ ]:
scores_data, agents = scores_analysis("4_EXP_All_Games", join=join, logs_path=logs_path)
stats_dict = {}
stats_dict["difference"] = {}
stats_dict["scores"] = {}
stats_dict["opponent_scores"] = {}

for player in agents:
   for opponent in agents:
      tempa = scores_data[scores_data["Player"]==player]
      tempb = scores_data[scores_data["Player"]==opponent]
      a = tempa["PlayerScore"]-tempa["OpponentScore"]
      b = tempb["PlayerScore"]-tempa["OpponentScore"]
      stats_dict("")
      tStat, pValue = stats.ttest_ind(a, b, equal_var = False) #run independent sample T-Test
      print("P-Value:{0} T-Statistic:{1}".format(pValue,tStat))

#t-test

In [ ]:
#t-test: https://www.hackdeploy.com/python-t-test-a-friendly-guide/

import pandas as pd
import os
import numpy as np
import math
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import ast
import random
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy as sp
from scipy import stats as spst
from datetime import date


print(os.path.abspath(os.getcwd()))

logs_path = "logs"
join = "/"
exp = "1_EXP_MCTS_Param"

def significance_test_allvsall(exp_name, threshold = 0.05):
   data =  pd.read_csv(logs_path + join + exp_name + join + "PlayerStats.csv")
   first = True
   lista = []
   for _ in range(len(data)):
      lista.append(first)
      if first: first = False
      else: first = True
   data["IsFirstPlayer"] = lista

   agents = data["Player"].unique()
   print("Agents: ", agents)

   ##Difference
   pvals = {}
   sig_dict = {}
   ##Scores
   s_pvals = {}
   s_sig_dict = {}
   ##2Scores
   s2_pvals = {}
   s2_sig_dict = {}
   checked_players = []
   filtered_data = data[data["IsFirstPlayer"]==True]
   #filtered_data = data
   for player in agents:
      player_significants = []
      player_pvals = []
      s_player_significants = []
      s_player_pvals = []
      s2_player_significants = []
      s2_player_pvals = []
      for opponent in agents:

         if player != opponent:
            filtered_data_a = filtered_data[filtered_data["Player"]==player]
            data_a = filtered_data_a[filtered_data_a["Opponent"]==opponent]
            a = data_a["PlayerScore"]-data_a["OpponentScore"]
            s2_a = data_a["OpponentScore"]
            s_a = data_a["PlayerScore"]

            filtered_data_b = filtered_data[filtered_data["Player"]==opponent]
            data_b = filtered_data_b[filtered_data_b["Opponent"]==player]
            b = data_b["PlayerScore"]-data_b["OpponentScore"]
            s2_b = data_b["OpponentScore"]
            s_b = data_b["PlayerScore"]
            
            tStat, pValue = spst.ttest_ind(a, b, equal_var = False)
            player_significants.append(pValue<threshold)
            player_pvals.append(pValue)
            #print("1:",player,", 2:",opponent," ,pval: ",pValue)

            s_tStat, s_pValue = spst.ttest_ind(s_a, s_b, equal_var = False)
            s_player_significants.append(s_pValue<threshold)
            s_player_pvals.append(s_pValue)

            s2_tStat, s2_pValue = spst.ttest_ind(s2_a, s2_b, equal_var = False)
            s2_player_significants.append(s2_pValue<threshold)
            s2_player_pvals.append(s2_pValue)

         else:
            player_significants.append(None)
            player_pvals.append(None)
            s_player_significants.append(None)
            s_player_pvals.append(None)
            s2_player_significants.append(None)
            s2_player_pvals.append(None)

      checked_players.append(player)

      sig_dict[player] = player_significants
      pvals[player] = player_pvals
      s_sig_dict[player] = s_player_significants
      s_pvals[player] = s_player_pvals
      s2_sig_dict[player] = s2_player_significants
      s2_pvals[player] = s2_player_pvals
   
   sig_data = pd.DataFrame(sig_dict, index=agents)
   pvals_data = pd.DataFrame(pvals, index=agents).round(5)
   s_sig_data = pd.DataFrame(s_sig_dict, index=agents)
   s_pvals_data = pd.DataFrame(s_pvals, index=agents).round(5)
   s2_sig_data = pd.DataFrame(s2_sig_dict, index=agents)
   s2_pvals_data = pd.DataFrame(s2_pvals, index=agents).round(5)

   today = date.today()
   d = today.strftime("_%d_%m_%Y_%H_%M_%S")

   sig_data.to_csv(logs_path + join + "analysis" + join + exp_name + "_diff_significance"+ d+".csv")
   pvals_data.to_csv(logs_path + join + "analysis" + join + exp_name + "_diff_pvalues"+ d+".csv")
   s_sig_data.to_csv(logs_path + join + "analysis" + join + exp_name + "_score_significance"+ d+".csv")
   s_pvals_data.to_csv(logs_path + join + "analysis" + join + exp_name + "_score_pvalues"+ d+".csv")
   s2_sig_data.to_csv(logs_path + join + "analysis" + join + exp_name + "_score2_significance"+ d+".csv")
   s2_pvals_data.to_csv(logs_path + join + "analysis" + join + exp_name + "_score2_pvalues"+ d+".csv")
      
   return pvals_data, s_pvals_data, s2_pvals_data

#pvals_data, s_pvals_data, s2_pvals_data = significance_test_allvsall(exp)
pvals_data, s_pvals_data, s2_pvals_data = significance_test_allvsall("2_EXP_RAVE_Param")
print(pvals_data)
print(s_pvals_data)
print(s2_pvals_data)


         




d:\Users\fredx\Documents\Gits\mcts_uct_fred
Agents:  ['M_RAVE_0.5' 'M_RAVE_3' 'M_RAVE_2' 'M_RAVE_1' 'M_RAVE_0.25'
 'Add_M_RAVE_sqrt2']
                  M_RAVE_0.5  M_RAVE_3  M_RAVE_2  M_RAVE_1  M_RAVE_0.25  \
M_RAVE_0.5               NaN   0.00000   0.00000   0.00002      0.22330   
M_RAVE_3             0.00000       NaN   0.00002   0.00000      0.00000   
M_RAVE_2             0.00000   0.00002       NaN   0.00000      0.00000   
M_RAVE_1             0.00002   0.00000   0.00000       NaN      0.00122   
M_RAVE_0.25          0.22330   0.00000   0.00000   0.00122          NaN   
Add_M_RAVE_sqrt2     0.00000   0.00000   0.00069   0.15605          NaN   

                  Add_M_RAVE_sqrt2  
M_RAVE_0.5                 0.00000  
M_RAVE_3                   0.00000  
M_RAVE_2                   0.00069  
M_RAVE_1                   0.15605  
M_RAVE_0.25                    NaN  
Add_M_RAVE_sqrt2               NaN  
                  M_RAVE_0.5  M_RAVE_3  M_RAVE_2  M_RAVE_1  M_RAVE_0.25  \
M_RAV

In [ ]:
#t-test: https://www.hackdeploy.com/python-t-test-a-friendly-guide/

import pandas as pd
import os
import numpy as np
import math
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import ast
import random
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy as sp
from scipy import stats as spst
from datetime import date


print(os.path.abspath(os.getcwd()))

logs_path = "logs"
join = "/"
exp = "1_EXP_MCTS_Param"

def significance_test_allvsall(exp_name, threshold = 0.05):
   data =  pd.read_csv(logs_path + join + exp_name + join + "PlayerStats.csv")
   first = True
   lista = []
   for _ in range(len(data)):
      lista.append(first)
      if first: first = False
      else: first = True
   data["IsFirstPlayer"] = lista
   #filtered_data = data[data["IsFirstPlayer"]==True]

   agents = data["Player"].unique()
   na = len(agents)
   print("Agents: ", agents)
   all_colors = ["blue","orange","green","yellow","red","black","pink","brown","purple","gray","goldenrod","magenta","#0AD","6C4" ]
   assert(na <= len(all_colors))
   agents_legends = [a+"_as_1st" for a in agents] + [a+"_as_2nd" for a in agents]
   colors = {a:all_colors[i] for i,a in enumerate(agents_legends)}

   ##Scores
   s_pvals = {}
   s_sig_dict = {}

   checked_players = []
   added_legends = []
   
   plot = make_subplots(rows=na+1, cols=na+1
            #,shared_xaxes=True
            ,shared_yaxes=True
            #,subplot_titles=["Opponent's combined score","Opponent's combined score difference","Opponent's combined score proportion"]
            #,row_heights=[4 for _ in exps_names]
            #,vertical_spacing=0.3
            ,print_grid=True
            ,x_title="Player"
            )
   

   #filtered_data = data
   to_plot = {name:[] for name in ["A1","A2","row","column","nr","nc"]}
   for nc, column in enumerate(agents): #the row agent
      s_player_significants = []
      s_player_pvals = []
      for nr, row in enumerate(agents): #the column agent
         if row != column:
            if row in checked_players: #if in the upper triangle of the matrix
               #el volteado
               group_separator = "_as_2nd"
               row_filtered_data = data[(data["IsFirstPlayer"]==False) & (data["Player"]==row) & (data["Opponent"]==column)] #S2,1
               row_scores = row_filtered_data["PlayerScore"]

               column_filtered_data = data[(data["IsFirstPlayer"]==False) & (data["Player"]==column) & (data["Opponent"]==row)] #S2,2
               column_scores = column_filtered_data["PlayerScore"]

            else:
               group_separator = "_as_1st"
               row_filtered_data = data[(data["IsFirstPlayer"]==True) & (data["Player"]==row) & (data["Opponent"]==column)] #S1,1
               row_scores = row_filtered_data["PlayerScore"]

               column_filtered_data = data[(data["IsFirstPlayer"]==True) & (data["Player"]==column) & (data["Opponent"]==row)] #S1,2
               column_scores = column_filtered_data["PlayerScore"]

            s_Stat, s_pValue = spst.ttest_ind(row_scores, column_scores, equal_var = False)
            s_player_significants.append(s_pValue<threshold)
            s_player_pvals.append(s_pValue)
            print("Upper: row: ", row, ", column:", column, ", lenghts:", str(len(row_scores)), str(len(column_scores)))
            row_legend = row+group_separator not in added_legends
            col_legend = column+group_separator not in added_legends
            plot.add_trace(
                go.Box(
                    y=row_scores,
                    x=[row for _ in range(len(row_scores))],
                    name=row+group_separator,
                    legendgroup=row+group_separator,
                    marker_color=colors[row+group_separator],
                    showlegend=row_legend
                    ),
                row = nr+1,
                col = nc+1
                )
            plot.add_trace(
                go.Box(
                    y=column_scores,
                    x=[column for _ in range(len(column_scores))],
                    name=column+group_separator,
                    legendgroup=column+group_separator,
                    marker_color=colors[column+group_separator],
                    showlegend=col_legend
                    ),
                row = nr+1,
                col = nc+1
                )
            added_legends = added_legends + [column+group_separator, row+group_separator]
            added_legends = list(set(added_legends))
         else:
            s_player_significants.append(None)
            s_player_pvals.append(None)
            #print("Upper: row: ", row, ", column:", column, ", lenghts:", None, None)
         

      checked_players.append(column)

      print("Adding column: ", column, ", values: ", s_player_pvals)
      s_sig_dict[column] = s_player_significants
      s_pvals[column] = s_player_pvals

   plot.update_layout(
      #boxmode='group'
      #,
      title_text="Scores distribution"
      #,xaxis_title="Player"
      ,yaxis_title="Score"
      #,legend_title="Player"
      ,autosize=False
      ,width=1000
      ,height=1000)
   #plot.update_xaxes(title_text = "")
   plot.show()
   

   s_sig_data = pd.DataFrame(s_sig_dict, index=agents)
   s_pvals_data = pd.DataFrame(s_pvals, index=agents).round(5)

   today = date.today()
   d = today.strftime("_%d_%m_%Y_%H_%M_%S")

   s_sig_data.to_csv(logs_path + join + "analysis" + join + exp_name + "_score_significance"+ d+".csv")
   s_pvals_data.to_csv(logs_path + join + "analysis" + join + exp_name + "_score_pvalues"+ d+".csv")
      
   return s_pvals_data



d:\Users\fredx\Documents\Gits\mcts_uct_fred


In [ ]:
out = significance_test_allvsall("3_EXP_Minimax")
#out = significance_test_allvsall("2_EXP_RAVE_Param")
#out = significance_test_allvsall("1_EXP_MCTS_Param")
print(out)

In [ ]:
import pandas as pd
import os
import numpy as np
import math
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import ast
import random
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy as sp
from scipy import stats as spst
from datetime import date


print(os.path.abspath(os.getcwd()))

logs_path = "logs"
join = "/"
#exp = "1_EXP_MCTS_Param"
#exp = "3_EXP_Minimax"
#exp = "2_EXP_RAVE_Param"
exp = "4_EXP_All_Games"

def gather_scores(exp_name, file_name = None):

   #get data
   if file_name is None:
      data =  pd.read_csv(logs_path + join + exp_name + join + "PlayerStats.csv")
   else:
      data =  pd.read_csv(logs_path + join + exp_name + join + file_name)
   
   #Pre-processing
   first = True
   lista = []
   for _ in range(len(data)):
      lista.append(first)
      if first: first = False
      else: first = True
   data["PlayerWasFirst"] = lista
   data['Player'] = data['Player'].str.replace('Add_','')
   data['Opponent'] = data['Opponent'].str.replace('Add_','')

   #prepare variables, order (not used currently)
   if exp_name=="1_EXP_MCTS_Param":
      agents = ["MCTS_0.25","MCTS_0.5","MCTS_1","MCTS_sqrt2","MCTS_2","MCTS_3"]
   else:
      agents = data["Player"].unique()
   print("Agents: ", agents)

   #calculations
   data["Difference"] = data["PlayerScore"]-data["OpponentScore"]
   data["CityScore"] = data["CompleteCityScore"]+data["IncompleteCityScore"]
   data["RoadScore"] = data["CompleteRoadScore"]+data["IncompleteRoadScore"]
   data["MonasteryScore"] = data["CompleteMonasteryScore"]+data["IncompleteMonasteryScore"]

   #filtering and grouping
   #t_data = data[data["PlayerWasFirst"]==True]
   data_split = data.loc[:,["Player","Opponent","PlayerScore","Difference","CityScore","RoadScore","MonasteryScore","Game","PlayerWasFirst"]]
   data_split = data_split.sort_values(by=["Player","Opponent"])
   
   data_isf = data.loc[:,["Player","Opponent","PlayerScore","Difference","CityScore","RoadScore","MonasteryScore","PlayerWasFirst"]]
   data = data.loc[:,["Player","Opponent","PlayerScore","Difference","CityScore","RoadScore","MonasteryScore"]]
   
   final_df = data.groupby(["Player","Opponent"]).agg([np.mean, np.std, np.var]).round(3)
   final_df_isf = data_isf.groupby(["Player","Opponent","PlayerWasFirst"]).agg([np.mean, np.std, np.var]).round(3)

   #save to a file
   today = date.today()
   d = today.strftime("_%d_%m_%Y_%H_%M_%S")

   #to file
   data_split.to_csv(logs_path + join + "analysis" + join + exp_name + "_scores"+ d+".csv", index=False)
   #final_df_isf.to_csv(logs_path + join + "analysis" + join + exp_name + "_gather_scores_isf"+ d+".csv")
   final_df.to_csv(logs_path + join + "analysis" + join + exp_name + "_gather_scores"+ d+".csv", index=False)
   final_df_isf.to_csv(logs_path + join + "analysis" + join + exp_name + "_gather_scores_isf"+ d+".csv", index=False)
      
   return final_df

df = gather_scores(exp, "ScoreAnalysis_PlayerStats.csv")
df.head()


d:\Users\fredx\Documents\Gits\mcts_uct_fred
Agents:  ['ES_B_MCTS' 'ES_B_S_MCTS' 'M_RAVE_0.25' 'Star1' 'Random' 'M_RAVE_0.5'
 'ES_MCTS' 'MCTS_0.5_400' 'MCTS_sq2_400' 'MCTS_0.52800' 'MCTS_sq2_2800'
 'M_RAVE_0.52800' 'M_RAVE_0.252800']


PlayerScore                  Difference          \
                               mean     std      var       mean     std   
Player    Opponent                                                        
ES_B_MCTS ES_B_S_MCTS        77.733  17.483  305.651     -0.933  22.221   
          ES_MCTS            90.967  13.229  174.999     43.067  20.184   
          MCTS_0.52800       77.367  16.311  266.033      1.067  19.172   
          MCTS_0.5_400       78.533  17.733  314.464      9.433  24.933   
          MCTS_sq2_2800      75.367  11.976  143.413    -12.967  19.455   

                                 CityScore                  RoadScore         \
                             var      mean     std      var      mean    std   
Player    Opponent                                                             
ES_B_MCTS ES_B_S_MCTS    493.789    33.133   9.726   94.602    16.767  7.454   
          ES_MCTS        407.375    45.867  10.543  111.154    17.867  7.123   
          MCTS_0.52800   367.582    31.633   9.817   96.378    19.500  5.824   
          MCTS_0.5_400   621.633    37.533  13.240  175.292    16.533  6.986   
          MCTS_sq2_2800  378.516    33.800   9.866   97.338    17.800  6.697   

                                MonasteryScore                 
                            var           mean    std     var  
Player    Opponent                                             
ES_B_MCTS ES_B_S_MCTS    55.564         10.033  8.728  76.171  
          ES_MCTS        50.740          9.933  7.007  49.099  
          MCTS_0.52800   33.914          7.933  7.492  56.133  
          MCTS_0.5_400   48.809          8.367  7.627  58.171  
          MCTS_sq2_2800  44.855          9.067  7.529  56.685

In [ ]:
###Plot bars with function complexities by turn


import pandas as pd
import os
import numpy as np
import math
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import ast
#import random
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy as sp
from scipy import stats as spst
from datetime import date

logs_path = "logs"
join = "/"
#exp = "1_EXP_MCTS_Param"
#exp = "3_EXP_Minimax"
exp = "2_EXP_RAVE_Param"

def formula_analysis(filtered_data, title, n_formulas=5, spaced="evenly", identifier = "black"):
    plot = go.Figure()
    x = filtered_data["Turn"]
    y = filtered_data["NumberNodes"]
    plot.add_trace(go.Box(
        name="Number of Nodes",x=x,y=y
        ,legendgroup='group1'
        ,showlegend=False
        ,marker_color = "black"
        ))
    plot.add_trace(go.Scatter(x=x, y=[13 for _ in x],
                    mode='lines',legendgroup='group1'
                    ,showlegend=False,
                    name='UCT'
                    ,marker_color = "black"))

    #select random formula
    functions = []
    turns = [int(max(x)/n_formulas)*i+1 for i in range(n_formulas)]
    for t in turns:
        temp_df = filtered_data[filtered_data["Turn"]==t]
        cont = True
        while(cont):
            random_index = np.random.randint(0,len(temp_df)-1)
            if temp_df["NumberNodes"].values[random_index] < 25 and temp_df["NumberNodes"].values[random_index] > 9:
                functions.append((t
                    ,temp_df["NumberNodes"].values[random_index]
                    ,temp_df["Function"].values[random_index]))
                cont = False
            #else: print("Finding another formula")

    markers = ["square", "diamond","cross","x","triangle-up","triangle-down","star","star-square","corcle-cross","pentagon"]
    for marker_id, f in enumerate(functions):#enumerate([[int(max(x)/n_formulas)*i+1] for i in range(n_formulas)]):
        x_formula = [f[0]]
        y_formula = [f[1]]
        formula_text = f[2]
        plot.add_trace(go.Scatter(
                        x=x_formula
                        ,y=y_formula
                        ,mode='markers'
                        ,legendgroup='group1'
                        ,showlegend=True
                        ,name='<b>'+formula_text+'</b>'#""#str(filtered_data["Function"].iloc[x_formula]).split("Name")[0]
                        ,marker_symbol = markers[marker_id] + "-open-dot"
                        #,marker_color = "black"
                        #,marker_size=9
                        ,marker=dict(
                            color=identifier,#"red",#'rgba(135, 206, 250, 0.5)',
                            size=10,
                            line=dict(
                                color='MediumPurple',
                                width=2.5
                            )
                        )))

    plot.update_layout(
                #title_text=title
                #,title_x=0.5
                #,title_y=1
                #xaxis_title="Turn"
                #,yaxis_title="Nodes"
                autosize=False
                ,width=700
                ,height=350
                ,plot_bgcolor='rgba(0,0,0,0)'
                ,legend=dict(
                    #title = "Formula",
                    #orientation="h",
                    #yanchor="top",
                    y=-0.65,
                    xanchor="center",
                    x=0.5,  
                    font = dict(family = "Arial", size = 14, color = "black"),
                    #bordercolor="LightSteelBlue",
                    borderwidth=2,
                    itemsizing='trace',
                    itemwidth = 30
                    )  
                )       
    plot['layout'].update(margin=dict(
                                        l=20,
                                        r=10,
                                        #b=0,
                                        t=10))    
    plot["layout"]["xaxis"]["range"] = [0.1, 37]  
    plot["layout"]["yaxis"]["tickfont"]["size"] = 13 ##dict(family = 'Old Standard TT, serif',size = 14,color = 'black'),
    plot["layout"]["yaxis"]["tickfont"]["color"] = "black"
    plot["layout"]["xaxis"]["tickfont"]["size"] = 13 ##dict(family = 'Old Standard TT, serif',size = 14,color = 'black'),
    plot["layout"]["xaxis"]["tickfont"]["color"] = "black"
    plot["layout"]["yaxis"]["tickmode"] = "linear"
    plot["layout"]["yaxis"]["tick0"] = 0
    plot["layout"]["yaxis"]["dtick"] = 20
    plot["layout"]["yaxis"]["showgrid"] = True
    plot["layout"]["yaxis"]["gridcolor"] = "black"
    plot["layout"]["yaxis"]["gridwidth"] = 0.6
    #plot["layout"]["xaxis"]["title"] = "Turn"
    #plot["layout"]["yaxis"]["title"] = "Nodes"
    #plot['layout'].legend.x = "Turn"
    #plot['layout'].legend.y = "Nodes"
    plot.update_xaxes(anchor="free"
                    ,position=0.04)
    plot.update_xaxes(showline=True, linewidth=1.5, linecolor='black', 
                  #zeroline=True, 
                  #zerolinewidth=1.5, zerolinecolor='black',

                )
    plot.update_yaxes(showline=True, linewidth=1.5, linecolor='black', mirror=True
                  #gridcolor='black', gridwidth=1,
                   #zeroline=True, 
                   #zerolinewidth=1.5, zerolinecolor='black',
    )
    plot.add_shape(
        # Rectangle with reference to the plot
            type="rect",
            xref="paper",
            yref="paper",
            x0=0,
            y0=1,
            x1=1.0,
            y1=1.0,
            line=dict(
                color="black",
                 width=1,
             )
         )
    
    plot.add_annotation(
        x=0,
        y=-0.05,
        xref="paper",
        yref="paper",
        text="Turns",
        showarrow=False,
        font=dict(
            #family="Courier New, monospace",
            size=12,
            color="black"
            ),
        align="left",
        #arrowhead=2,
        #arrowsize=1,
        #arrowwidth=2,
        #arrowcolor="#636363",
        ax=20,
        ay=-30,
        #bordercolor="black",
        #borderwidth=2,
        #borderpad=4,
        #bgcolor="#ff7f0e",
        #opacity=0.8
        )
    plot.add_annotation(
        x=-0.03,
        y=0.05,
        xref="paper",
        yref="paper",
        text="Nodes",
        showarrow=False,
        font=dict(
            #family="Courier New, monospace",
            size=12,
            color="black"
            ),
        align="left",
        #arrowhead=2,
        #arrowsize=1,
        #arrowwidth=2,
        #arrowcolor="#636363",
        ax=20,
        ay=-30,
        #bordercolor="black",
        #borderwidth=2,
        #borderpad=4,
        #bgcolor="#ff7f0e",
        #opacity=0.8
        textangle=-90
        )
    
    plot.show()
    plot.write_image(agent_name + identifier + ".svg", width=1920, height=1080)
    #plot.write_image(title+ ".pdf")


np.random.seed(13)
full_df =  pd.read_csv(logs_path + join + "4_EXP_All_Games" + join + "CollectedEvolvedUCT.csv")
agents = list(full_df["Name"].unique())
for agent in agents:
    filtered_df = full_df[full_df["Name"]==agent]
    if agent == "ES_B_S_MCTS": 
        agent_name = "SIEA_MCTS"
        #np.random.seed(2)
    if agent == "ES_B_MCTS": 
        agent_name = "EA_MCTS"
        #np.random.seed(3)ff
    if agent == "ES_MCTS": 
        agent_name = "EA_p_MCTS"
        #np.random.seed(4)
    formula_analysis(filtered_df, agent_name, identifier = "black")

In [ ]:
import pandas as pd
import os
import numpy as np
import math
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import ast
import random
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy as sp
from scipy import stats as spst
from datetime import date

logs_path = "logs"
join = "/"

def create_league_table(data, exp_name):
   #pre-processing
   first = True
   lista = []
   for _ in range(len(data)):
      lista.append(first)
      if first: first = False
      else: first = True
   data["PlayerWasFirst"] = lista
   data['Player'] = data['Player'].str.replace('Add_','')
   data['Opponent'] = data['Opponent'].str.replace('Add_','')
   data['Differences'] = data['PlayerScore']-data["OpponentScore"]

   dict_holder = defaultdict(lambda:[])
   agents = data["Player"].unique()
   for agent in agents:
      filetered_data = data[data["Player"]==agent]
      dict_holder["MatchesPlayed"].append(len(filetered_data[filetered_data["Game"]==1]))
      dict_holder["Player"].append(agent)
      ds = 0
      ws = 0
      ls = 0
      points = 0
      bwp = 0
      blp = 0
      pds = 0
      for opponent in agents:
         if agent != opponent:
            for isfirst in [True, False]:
               uniques_filetered_data = filetered_data[filetered_data["PlayerWasFirst"]==isfirst]
               opponent_filtered_data = uniques_filetered_data[uniques_filetered_data["Opponent"]==opponent]
               if len(opponent_filtered_data) == 0:
                  print(agent, "as first:",str(isfirst), "vs ", opponent, ": ",len(opponent_filtered_data))
               else:
                  pds += round(sum(opponent_filtered_data["Differences"])/len(opponent_filtered_data),3)
                  wins = len(opponent_filtered_data[opponent_filtered_data["Win"]==1])
                  losses = len(opponent_filtered_data[opponent_filtered_data["Loss"]==1])
                  if wins > losses:
                     ws += 1
                     points += 4
                     if (wins/(wins+losses) >= 0.75):
                        points += 1
                        bwp += 1
                  elif losses > wins:
                     ls += 1
                     if (losses/(wins+losses) <= 0.55):
                        points += 1
                        blp += 1
                  else:
                     ds += 1
                     points += 2
      dict_holder["Points"].append(points)
      dict_holder["BWP"].append(bwp)
      dict_holder["BLP"].append(blp)
      dict_holder["W"].append(ws)
      dict_holder["L"].append(ls)
      dict_holder["D"].append(ds)
      dict_holder["PD"].append(pds)
      #for opponent in agents:
      #   opponent_filtered_data = filetered_data[filetered_data["Opponent"]==opponent]
      #agent_dict["BWP"] = sum(filetered_data["Differences"])


   final_df = pd.DataFrame(dict_holder)
   final_df.sort_values(by=["Points","PD"])
   today = date.today()
   d = today.strftime("_%d_%m_%Y_%H_%M_%S")
   final_df.to_csv(logs_path + join + "analysis" + join + exp_name + "_NewLeagueTable"+ d+".csv")

exp_name = "4_EXP_All_Games"
output_name = "LAST"
full_df =  pd.read_csv(logs_path + join + exp_name + join + "ScoreAnalysis_PlayerStats.csv")
create_league_table(full_df, output_name)
   